In [ ]:
import config

In [ ]:
#Import necessary libraries
import ibm_db
from ibm_db import connect
import ibm_db_dbi
import ibm_db_sa
import sqlalchemy
import requests
import numpy as np
import pandas as pd
from pandas import DataFrame
from pandas import json_normalize
import shapely
import fiona
from shapely import geos
import geopandas as gpd
from geopandas import GeoDataFrame
import matplotlib.pyplot as plt
import descartes
from descartes.patch import PolygonPatch
import pathlib
import pyproj
import rtree
import folium
from folium import GeoJson
from folium import LayerControl
from shapely.geometry import Point
from folium import plugins
import json
from branca.colormap import linear

In [ ]:
import seaborn as sns
from scipy import stats

In [ ]:
%run db2.ipynb

In [ ]:
dsn = (
    "DRIVER={0};"
    "DATABASE={1};"
    "HOSTNAME={2};"
    "PORT={3};"
    "PROTOCOL={4};"
    "UID={5};"
    "PWD={6};").format(config.dsn_driver, config.dsn_database, 
                       config.dsn_hostname, config.dsn_port, 
                       config.dsn_protocol, config.dsn_uid, config.dsn_pwd)

try:
    conn = ibm_db.connect(dsn, "", "")
    print ("Connected to database")

except:
    print ("Unable to connect: ", ibm_db.conn_errormsg() )

In [ ]:
%load_ext sql

In [ ]:
def connect_sql (dsn_uid, dsn_sql_pwd, dsn_hostname, dsn_port, dsn_database):
    %sql ibm_db_sa://{dsn_uid}:{dsn_sql_pwd}{dsn_hostname}:{dsn_port}/{dsn_database}

In [ ]:
connect_sql (dsn_uid=config.dsn_uid, 
             dsn_sql_pwd=config.dsn_sql_pwd, 
             dsn_hostname=config.dsn_hostname,
             dsn_port=config.dsn_port,  
             dsn_database=config.dsn_database)

In [ ]:
#Close connection
#ibm_db.close(conn)
#Uncomment the line above this and rerun the cell to close the connection

---
## Defining Functions to Use Throughout
---

In [ ]:
#Define Equation to get Population Density
def popdens(x, y):
    return (x / (y/1000000))

In [ ]:
#Define Equation to get Percentage of Race Population
def raceperc(x,y):
    return ((x / y)*100)

In [ ]:
#Define Equation to get Percentage of Remaining Race Population
def inverseraceperc(v,x,y,z):
    return (100-(v+x+y+z))

In [ ]:
#Define Equation to use Margins of Error
def upperme (x,z): 
    return x+z
def lowerme (x,z): 
    return x-z

---
## Place Level: 

##### Importing and Cleaning Shapefile
---

In [ ]:
#Import shape file of places
place_shp_file = ('nhgis0008_shape/nhgis0008_shapefile_tl2019_us_place_2019/US_place_2019.shp')

#Read in shape file to geopandas
place = gpd.read_file(place_shp_file)

In [ ]:
#Check Place Projection System
place.crs

In [ ]:
#Take an Overarching Look at Place df 
place.info()

In [ ]:
#Drop Columns that are deemed irrelevant 
place=place.drop(columns=['PLACEFP', 'PLACENS', 'GEOID', 'NAMELSAD', 'LSAD', 
                          'CLASSFP', 'PCICBSA', 'PCINECTA', 'MTFCC', 'FUNCSTAT'])

In [ ]:
#Rename Columns to be More Descriptive 
place.rename(columns = {'STATEFP':'State FIPS Code', 
                         'NAME':'Place Name', 
                         'ALAND':'Land Area', 
                         'AWATER':'Water Area', 
                         'INTPTLAT':'Place Center Lat', 
                         'INTPTLON':'Place Center Long', 
                         'Shape_Leng':'Place Perimeter', 
                         'Shape_Area':'Place Area'}, inplace = True) 

In [ ]:
#Take another Overarching Look at Place df 
place.info()

---
## Place Level: 

##### Importing Data with SQL and Cleaning
---

In [ ]:
#Import Place Data from DB2 Using SQL Magic
PLACE_DATA = %sql select * from PLACE1 where STATE LIKE 'Texas%'; 

In [ ]:
#Create DataFrame for Imported data
place_df = PLACE_DATA.DataFrame()
place_df.head()

In [ ]:
#Take an Overarching Look at place_df
place_df.info()

In [ ]:
#Rename Columns
place_df.rename({'gisjoin': 'GISJOIN',
                 'statea': 'State FIPS Code',
                 'countya': 'County FIPS Code',
                 'tracta': 'Tract FIPS Code',
                 'name_e': 'Place Area Name',
                 'alube001': 'Total Place Pop',
                 'alt1e001': 'Total Place Median Age',
                 'alt1e002': 'Place Median Age M',
                 'alt1e003': 'Place Median Age F',
                 'aluce002': 'Place Pop White',
                 'aluce003': 'Place Pop Black or African American',
                 'aluce004': 'Place Pop American Indian and Alaska Native',
                 'aluce005': 'Place Pop Asian',
                 'alx5e001': 'Place Per Capita Income - 12 Months',
                 'alubm001': 'Total Place Pop ME',
                 'alt1m001': 'Total Place Median Age ME',
                 'alt1m002': 'Place Median Age ME M',
                 'alt1m003': 'Place Median Age ME F',
                 'alucm002': 'Place Pop ME White',
                 'alucm003': 'Place Pop ME Black or African American',
                 'alucm004': 'Place Pop ME American Indian and Alaska Native',
                 'alucm005': 'Place Pop ME Asian',
                 'alx5m001': 'Place Per Capita Income ME - 12 Months'}, axis = "columns", inplace = True) 

In [ ]:
#Trim to Columns of Interest
place_df=place_df[['GISJOIN','State FIPS Code', 'Place Area Name', 
                      'Total Place Pop', 'Total Place Median Age', 'Place Median Age M', 'Place Median Age F',
                      'Place Pop White', 'Place Pop Black or African American', 
                      'Place Pop American Indian and Alaska Native', 'Place Pop Asian', 
                      'Place Per Capita Income - 12 Months', 'Total Place Pop ME', 
                      'Total Place Median Age ME', 'Place Median Age ME M', 'Place Median Age ME F',
                      'Place Pop ME White', 'Place Pop ME Black or African American', 
                      'Place Pop ME American Indian and Alaska Native', 'Place Pop ME Asian', 
                      'Place Per Capita Income ME - 12 Months']]

In [ ]:
#Convert Age columns to Floats
place_df['Total Place Median Age']=pd.to_numeric(place_df['Total Place Median Age'],errors='coerce',downcast='float')
place_df['Place Median Age M']=pd.to_numeric(place_df['Place Median Age M'],errors='coerce',downcast='float')
place_df['Place Median Age F']=pd.to_numeric(place_df['Place Median Age F'],errors='coerce',downcast='float')
place_df['Total Place Median Age ME']=pd.to_numeric(place_df['Total Place Median Age ME'],errors='coerce',downcast='float')
place_df['Place Median Age ME M']=pd.to_numeric(place_df['Place Median Age ME M'],errors='coerce',downcast='float')
place_df['Place Median Age ME F']=pd.to_numeric(place_df['Place Median Age ME F'],errors='coerce',downcast='float')

In [ ]:
#Final Overarching Look
place_df.info()

---
## Place Level: 

##### Merging Data with GIS & Adding in Columns Using Functions
---

In [ ]:
#Merge Data with GIS Data
place_merge = place.merge(place_df, on='GISJOIN')
place_merge

In [ ]:
#Create Column for Population Density
place_merge["Pop Density per sq km"] = popdens(place_merge["Total Place Pop"], place_merge["Land Area"])
place_merge.head()

In [ ]:
#Create Columns for Percentage Breakdown of Race

place_merge["Place Percentage White"] = raceperc(place_merge["Place Pop White"], 
                                           place_merge["Total Place Pop"])

place_merge["Place Percentage Black or African American"] = raceperc(place_merge["Place Pop Black or African American"], 
                                                               place_merge["Total Place Pop"])

place_merge["Place Percentage American Indian and Alaskan Native"] = raceperc(place_merge["Place Pop American Indian and Alaska Native"], 
                                                                        place_merge["Total Place Pop"])

place_merge["Place Percentage Asian"] = raceperc(place_merge["Place Pop Asian"], 
                                           place_merge["Total Place Pop"])

place_merge["Place Percentage Other Race"] = inverseraceperc(place_merge["Place Percentage White"], 
                                                       place_merge["Place Percentage Black or African American"], 
                                                       place_merge["Place Percentage American Indian and Alaskan Native"],
                                                       place_merge["Place Percentage Asian"])

In [ ]:
#Create Income String Columns for tooltips to display in Folium

#Cast numeric Income value as String
place_merge["Place Income String"] = place_merge['Place Per Capita Income - 12 Months'].astype(str)

#Find upper bound of Margin of Error and cast that to String
place_merge["Place Income ME Upper"] = upperme(place_merge["Place Per Capita Income - 12 Months"],
                                           place_merge["Place Per Capita Income ME - 12 Months"])
place_merge["Place Income ME Upper String"] = place_merge['Place Income ME Upper'].astype(str)

#Find lower bound of Margin of Error and cast that to String
place_merge["Place Income ME Lower"] = lowerme(place_merge["Place Per Capita Income - 12 Months"],
                                           place_merge["Place Per Capita Income ME - 12 Months"])
place_merge["Place Income ME Lower String"] = place_merge['Place Income ME Lower'].astype(str)

#Fill sentence set to display with Income, Income Margin of Error Range
place_merge["Place_Map_Income_Label"] = 'Place Income Estimate: $' + place_merge["Place Income String"] + ' (The US Census Bureau is 90% confident that the true per Capita Income in 2019 was between $' + place_merge["Place Income ME Lower String"] + ' and $' + place_merge["Place Income ME Upper String"] + ')'

In [ ]:
#Create Pop Density String Columns for tooltips to display in Folium

#Cast numeric Income value as String
place_merge["Place Pop Density String"] = place_merge['Pop Density per sq km'].astype(str)

#Fill sentence set to display with Pop Density
place_merge["Place_Map_Pop_Density_Label"] = 'Place Population Density: ' + place_merge["Place Pop Density String"] + ' per sq km'

In [ ]:
#Convert to 4326 Projection, will merge with city specific shapefiles later, and want projection system to match
place_merge_4326 = place_merge.to_crs(epsg='4326')

---
## Place Level: 

##### *Filtering to Houston*
---

In [ ]:
#Filter to Houston, TX values only
houston_place = place.loc[place['Place Name'] == 'Houston']
houston_place
houston_place = houston_place.loc[houston_place['State FIPS Code'] == '48']
houston_place

In [ ]:
#Convert to 4326 Projection for merge
houston_place_4326 = houston_place.to_crs(epsg='4326')

In [ ]:
#Merge shapes using inner & within methods
#Inner uses intersection of keys from both dfs, but retains only the left geometry 
#Within returns values that only intersect with the interior of the larger geometry 
houston_join_place_4326 = gpd.sjoin(place_merge_4326,houston_place_4326,how='inner',op='within')

In [ ]:
#Check for Nan/Null values
sum(houston_join_place_4326.isnull().any())

In [ ]:
#Print Nan/Null columns
nan_cols_place = [i for i in houston_join_place_4326.columns if houston_join_place_4326[i].isnull().any()]
print(nan_cols_place)

In [ ]:
#Print percentage of column that is Nan/Null values
houston_join_place_4326[houston_join_place_4326.columns[houston_join_place_4326.isnull().any()]].isnull().sum() * 100 / houston_join_place_4326.shape[0]

In [ ]:
#Drop Nan/Null Values
#houston_join_place_4326=houston_join_place_4326.dropna()

In [ ]:
#Save File as GeoJSON
houston_join_place_4326.to_file("houston_join_place_4326_gjson.geojson", driver='GeoJSON')

In [ ]:
#Create Function to Read GeoJSON Back in
houston_join_place_4326_gjson = gpd.read_file("houston_join_place_4326_gjson.geojson", driver='GeoJSON')

---
## Place Level: 

##### *Filtering to Dallas*
---

In [ ]:
#Filter to Dallas, TX values only
dallas_place = place.loc[place['Place Name'] == 'Dallas']
dallas_place
dallas_place = dallas_place.loc[dallas_place['State FIPS Code'] == '48']
dallas_place

In [ ]:
#Convert to 4326 Projection for merge
dallas_place_4326 = dallas_place.to_crs(epsg='4326')

In [ ]:
#Merge shapes using inner & within methods
#Inner uses intersection of keys from both dfs, but retains only the left geometry 
#Within returns values that only intersect with the interior of the larger geometry 
dallas_join_place_4326 = gpd.sjoin(place_merge_4326,dallas_place_4326,how='inner',op='within')

In [ ]:
#Check for Nan/Null values
sum(dallas_join_place_4326.isnull().any())

In [ ]:
#Print Nan/Null columns
nan_cols_place = [i for i in dallas_join_place_4326.columns if dallas_join_place_4326[i].isnull().any()]
print(nan_cols_place)

In [ ]:
#Print percentage of column that is Nan/Null values
dallas_join_place_4326[dallas_join_place_4326.columns[dallas_join_place_4326.isnull().any()]].isnull().sum() * 100 / dallas_join_place_4326.shape[0]

In [ ]:
#Drop Nan/Null Values
#dallas_join_place_4326=dallas_join_place_4326.dropna()

In [ ]:
#Save File as GeoJSON
dallas_join_place_4326.to_file("dallas_join_place_4326_gjson.geojson", driver='GeoJSON')

In [ ]:
#Create Function to Read GeoJSON Back in
dallas_join_place_4326_gjson = gpd.read_file("dallas_join_place_4326_gjson.geojson", driver='GeoJSON')

---
## County Level: 

##### Importing and Cleaning Shapefile
---

In [ ]:
#Import shape file of countys
county_shp_file = ('nhgis0008_shape/nhgis0008_shapefile_tl2019_us_county_2019/US_county_2019.shp')

#Read in shape file to geopandas
county = gpd.read_file(county_shp_file)

In [ ]:
#Check County Projection System
county.crs

In [ ]:
#Take an Overarching Look at County df 
county.info()

In [ ]:
#Drop Columns that are deemed irrelevant 
county=county.drop(columns=['STATEFP', 'COUNTYNS', 'GEOID', 'NAME', 'NAMELSAD', 'LSAD', 
                            'CLASSFP', 'MTFCC', 'CSAFP', 'CBSAFP', 'METDIVFP', 'FUNCSTAT'])

In [ ]:
#Rename Columns to be More Descriptive 
county.rename(columns = {'COUNTYFP':'County FIPS Code', 
                        'ALAND':'Land Area', 
                        'AWATER':'Water Area', 
                        'INTPTLAT':'County Center Lat', 
                        'INTPTLON':'County Center Long', 
                        'Shape_Leng':'County Perimeter', 
                        'Shape_Area': 'County Area'}, inplace = True) 

In [ ]:
#Take another Overarching Look at County df 
county.info()

---
## County Level: 

##### Importing Data with SQL and Cleaning
---

In [ ]:
#Import County Data from DB2 Using SQL Magic
COUNTY_DATA = %sql select * from COUNTY1 where STATE LIKE 'Texas%'; 

In [ ]:
#Create DataFrame for Imported data
county_df = COUNTY_DATA.DataFrame()
county_df.head()

In [ ]:
#Take an Overarching Look at county_df
county_df.info()

In [ ]:
#Rename Columns
county_df.rename({'gisjoin': 'GISJOIN',
                 'statea': 'State FIPS Code',
                 'county': 'County Name',
                 'countya': 'County FIPS Code',
                 'tracta': 'Tract FIPS Code',
                 'name_e': 'County Area Name',
                 'alube001': 'Total County Pop',
                 'alt1e001': 'Total County Median Age',
                 'alt1e002': 'County Median Age M',
                 'alt1e003': 'County Median Age F',
                 'aluce002': 'County Pop White',
                 'aluce003': 'County Pop Black or African American',
                 'aluce004': 'County Pop American Indian and Alaska Native',
                 'aluce005': 'County Pop Asian',
                 'alx5e001': 'County Per Capita Income - 12 Months',
                 'alubm001': 'Total County Pop ME',
                 'alt1m001': 'Total County Median Age ME',
                 'alt1m002': 'County Median Age ME M',
                 'alt1m003': 'County Median Age ME F',
                 'alucm002': 'County Pop ME White',
                 'alucm003': 'County Pop ME Black or African American',
                 'alucm004': 'County Pop ME American Indian and Alaska Native',
                 'alucm005': 'County Pop ME Asian',
                 'alx5m001': 'County Per Capita Income ME - 12 Months'}, axis = "columns", inplace = True) 

In [ ]:
#Trim to Columns of Interest
county_df=county_df[['GISJOIN', 'State FIPS Code', 'County Name', 'County FIPS Code', 'County Area Name', 
                      'Total County Pop', 'Total County Median Age', 'County Median Age M', 'County Median Age F',
                      'County Pop White', 'County Pop Black or African American', 
                      'County Pop American Indian and Alaska Native', 'County Pop Asian', 
                      'County Per Capita Income - 12 Months', 'Total County Pop ME', 
                      'Total County Median Age ME', 'County Median Age ME M', 'County Median Age ME F',
                      'County Pop ME White', 'County Pop ME Black or African American', 
                      'County Pop ME American Indian and Alaska Native', 'County Pop ME Asian', 
                      'County Per Capita Income ME - 12 Months']]

In [ ]:
#Convert Age columns to Floats
county_df['Total County Median Age']=pd.to_numeric(county_df['Total County Median Age'],errors='coerce',downcast='float')
county_df['County Median Age M']=pd.to_numeric(county_df['County Median Age M'],errors='coerce',downcast='float')
county_df['County Median Age F']=pd.to_numeric(county_df['County Median Age F'],errors='coerce',downcast='float')
county_df['Total County Median Age ME']=pd.to_numeric(county_df['Total County Median Age ME'],errors='coerce',downcast='float')
county_df['County Median Age ME M']=pd.to_numeric(county_df['County Median Age ME M'],errors='coerce',downcast='float')
county_df['County Median Age ME F']=pd.to_numeric(county_df['County Median Age ME F'],errors='coerce',downcast='float')

In [ ]:
#Final Overarching Look
county_df.info()

---
## County Level: 

##### Merging Data with GIS & Adding in Columns Using Functions
---

In [ ]:
#Merge Data with GIS Data
county_merge = county.merge(county_df, on='GISJOIN')
county_merge

In [ ]:
#Create Column for Population Density
county_merge["Pop Density per sq km"] = popdens(county_merge["Total County Pop"], county_merge["Land Area"])
county_merge.head()

In [ ]:
#Create Columns for Percentage Breakdown of Race

county_merge["County Percentage White"] = raceperc(county_merge["County Pop White"], 
                                           county_merge["Total County Pop"])

county_merge["County Percentage Black or African American"] = raceperc(county_merge["County Pop Black or African American"], 
                                                               county_merge["Total County Pop"])

county_merge["County Percentage American Indian and Alaskan Native"] = raceperc(county_merge["County Pop American Indian and Alaska Native"], 
                                                                        county_merge["Total County Pop"])

county_merge["County Percentage Asian"] = raceperc(county_merge["County Pop Asian"], 
                                           county_merge["Total County Pop"])

county_merge["County Percentage Other Race"] = inverseraceperc(county_merge["County Percentage White"], 
                                                       county_merge["County Percentage Black or African American"], 
                                                       county_merge["County Percentage American Indian and Alaskan Native"],
                                                       county_merge["County Percentage Asian"])

In [ ]:
#Create Income String Columns for tooltips to display in Folium

#Cast numeric Income value as String
county_merge["County Income String"] = county_merge['County Per Capita Income - 12 Months'].astype(str)

#Find upper bound of Margin of Error and cast that to String
county_merge["County Income ME Upper"] = upperme(county_merge["County Per Capita Income - 12 Months"],
                                           county_merge["County Per Capita Income ME - 12 Months"])
county_merge["County Income ME Upper String"] = county_merge['County Income ME Upper'].astype(str)

#Find lower bound of Margin of Error and cast that to String
county_merge["County Income ME Lower"] = lowerme(county_merge["County Per Capita Income - 12 Months"],
                                           county_merge["County Per Capita Income ME - 12 Months"])
county_merge["County Income ME Lower String"] = county_merge['County Income ME Lower'].astype(str)

#Fill sentence set to display with Income, Income Margin of Error Range
county_merge["County_Map_Income_Label"] = 'County Income Estimate: $' + county_merge["County Income String"] + ' (The US Census Bureau is 90% confident that the true per Capita Income in 2019 was between $' + county_merge["County Income ME Lower String"] + ' and $' + county_merge["County Income ME Upper String"] + ')'

In [ ]:
#Create Pop Density String Columns for tooltips to display in Folium

#Cast numeric Income value as String
county_merge["County Pop Density String"] = county_merge['Pop Density per sq km'].astype(str)

#Fill sentence set to display with Pop Density
county_merge["County_Map_Pop_Density_Label"] = 'County Population Density: ' + county_merge["County Pop Density String"] + ' per sq km'

In [ ]:
#Convert to 4326 Projection, will merge with city specific shapefiles later, and want projection system to match
county_merge_4326 = county_merge.to_crs(epsg='4326')

---
## County Level: 

##### *Filtering to Houston*
---

In [ ]:
#Merge shapes using inner & intersects methods
#Inner uses intersection of keys from both dfs, but retains only the left geometry 
#intersects returns values that intersect with either the interior or exterior of the larger geometry 
houston_join_county_4326 = gpd.sjoin(county_merge_4326,houston_place_4326,how='inner',op='intersects')

In [ ]:
#Check for Nan/Null values
sum(houston_join_county_4326.isnull().any())

In [ ]:
#Print Nan/Null columns
nan_cols_county = [i for i in houston_join_county_4326.columns if houston_join_county_4326[i].isnull().any()]
print(nan_cols_county)

In [ ]:
#Print percentage of column that is Nan/Null values
houston_join_county_4326[houston_join_county_4326.columns[houston_join_county_4326.isnull().any()]].isnull().sum() * 100 / houston_join_county_4326.shape[0]

In [ ]:
#Drop Nan/Null Values
#houston_join_county_4326=houston_join_county_4326.dropna()

In [ ]:
#Save File as GeoJSON
houston_join_county_4326.to_file("houston_join_county_4326_gjson.geojson", driver='GeoJSON')

In [ ]:
#Create Function to Read GeoJSON Back in
houston_join_county_4326_gjson = gpd.read_file("houston_join_county_4326_gjson.geojson", driver='GeoJSON')

---
## County Level: 

##### *Filtering to Dallas*
---

In [ ]:
#Merge shapes using inner & intersects methods
#Inner uses intersection of keys from both dfs, but retains only the left geometry 
#intersects returns values that intersect with either the interior or exterior of the larger geometry 
dallas_join_county_4326 = gpd.sjoin(county_merge_4326,dallas_place_4326,how='inner',op='intersects')

In [ ]:
#Check for Nan/Null values
sum(dallas_join_county_4326.isnull().any())

In [ ]:
#Print Nan/Null columns
nan_cols_county = [i for i in dallas_join_county_4326.columns if dallas_join_county_4326[i].isnull().any()]
print(nan_cols_county)

In [ ]:
#Print percentage of column that is Nan/Null values
dallas_join_county_4326[dallas_join_county_4326.columns[dallas_join_county_4326.isnull().any()]].isnull().sum() * 100 / dallas_join_county_4326.shape[0]

In [ ]:
#Drop Nan/Null Values
#dallas_join_county_4326=dallas_join_county_4326.dropna()

In [ ]:
#Save File as GeoJSON
dallas_join_county_4326.to_file("dallas_join_county_4326_gjson.geojson", driver='GeoJSON')

In [ ]:
#Create Function to Read GeoJSON Back in
dallas_join_county_4326_gjson = gpd.read_file("dallas_join_county_4326_gjson.geojson", driver='GeoJSON')

---
## Tract Level: 

##### Importing and Cleaning Shapefile
---

In [ ]:
#Import shape file of tracts
tract_shp_file = ('nhgis0008_shape/nhgis0008_shapefile_tl2019_us_tract_2019/US_tract_2019.shp')

#Read in shape file to geopandas
tract = gpd.read_file(tract_shp_file)

In [ ]:
#Check Tract Projection System
tract.crs

In [ ]:
#Take an Overarching Look at Tract df 
tract.info()

In [ ]:
#Drop Columns that are deemed irrelevant 
tract=tract.drop(columns=['STATEFP', 'COUNTYFP', 'TRACTCE', 'GEOID', 'NAME', 'NAMELSAD', 'MTFCC', 'FUNCSTAT'])

In [ ]:
#Rename Columns to be More Descriptive 
tract.rename(columns = {'ALAND':'Land Area', 
                        'AWATER':'Water Area', 
                        'INTPTLAT':'Tract Center Lat', 
                        'INTPTLON':'Tract Center Long', 
                        'Shape_Leng':'Tract Perimeter', 
                        'Shape_Area': 'Tract Area'}, inplace = True) 

In [ ]:
#Take another Overarching Look at Tract df 
tract.info()

---
## Tract Level: 

##### Importing Data with SQL and Cleaning
---

In [ ]:
#Import Tract Data from DB2 Using SQL Magic
TRACT_DATA = %sql select * from TRACT1 where STATE LIKE 'Texas%'; 

In [ ]:
#Create DataFrame for Imported data
tract_df = TRACT_DATA.DataFrame()
tract_df.head()

In [ ]:
#Take an Overarching Look at tract_df
tract_df.info()

In [ ]:
#Rename Columns
tract_df.rename({'gisjoin': 'GISJOIN',
                 'statea': 'State FIPS Code',
                 'county': 'County Name',
                 'countya': 'County FIPS Code',
                 'tracta': 'Tract FIPS Code',
                 'name_e': 'Tract Area Name',
                 'alube001': 'Total Tract Pop',
                 'alt1e001': 'Total Tract Median Age',
                 'alt1e002': 'Tract Median Age M',
                 'alt1e003': 'Tract Median Age F',
                 'aluce002': 'Tract Pop White',
                 'aluce003': 'Tract Pop Black or African American',
                 'aluce004': 'Tract Pop American Indian and Alaska Native',
                 'aluce005': 'Tract Pop Asian',
                 'alx5e001': 'Tract Per Capita Income - 12 Months',
                 'alubm001': 'Total Tract Pop ME',
                 'alt1m001': 'Total Tract Median Age ME',
                 'alt1m002': 'Tract Median Age ME M',
                 'alt1m003': 'Tract Median Age ME F',
                 'alucm002': 'Tract Pop ME White',
                 'alucm003': 'Tract Pop ME Black or African American',
                 'alucm004': 'Tract Pop ME American Indian and Alaska Native',
                 'alucm005': 'Tract Pop ME Asian',
                 'alx5m001': 'Tract Per Capita Income ME - 12 Months'}, axis = "columns", inplace = True) 

In [ ]:
#Trim to Columns of Interest
tract_df=tract_df[['GISJOIN', 'State FIPS Code', 'County Name', 'County FIPS Code', 'Tract FIPS Code', 
                   'Tract Area Name', 'Total Tract Pop', 'Total Tract Median Age', 'Tract Median Age M', 
                   'Tract Median Age F', 'Tract Pop White', 'Tract Pop Black or African American', 
                   'Tract Pop American Indian and Alaska Native', 'Tract Pop Asian', 
                   'Tract Per Capita Income - 12 Months', 'Total Tract Pop ME', 
                   'Total Tract Median Age ME', 'Tract Median Age ME M', 'Tract Median Age ME F',
                   'Tract Pop ME White', 'Tract Pop ME Black or African American', 
                   'Tract Pop ME American Indian and Alaska Native', 'Tract Pop ME Asian', 
                   'Tract Per Capita Income ME - 12 Months']]

In [ ]:
#Convert Age columns to Floats
tract_df['Total Tract Median Age']=pd.to_numeric(tract_df['Total Tract Median Age'],errors='coerce',downcast='float')
tract_df['Tract Median Age M']=pd.to_numeric(tract_df['Tract Median Age M'],errors='coerce',downcast='float')
tract_df['Tract Median Age F']=pd.to_numeric(tract_df['Tract Median Age F'],errors='coerce',downcast='float')
tract_df['Total Tract Median Age ME']=pd.to_numeric(tract_df['Total Tract Median Age ME'],errors='coerce',downcast='float')
tract_df['Tract Median Age ME M']=pd.to_numeric(tract_df['Tract Median Age ME M'],errors='coerce',downcast='float')
tract_df['Tract Median Age ME F']=pd.to_numeric(tract_df['Tract Median Age ME F'],errors='coerce',downcast='float')

In [ ]:
#Final Overarching Look
tract_df.info()

---
## Tract Level: 

##### Merging Data with GIS & Adding in Columns Using Functions
---

In [ ]:
#Merge Data with GIS Data
tract_merge = tract.merge(tract_df, on='GISJOIN')
tract_merge

In [ ]:
#Create Column for Population Density
tract_merge["Pop Density per sq km"] = popdens(tract_merge["Total Tract Pop"], tract_merge["Land Area"])
tract_merge.head()

In [ ]:
#Create Columns for Percentage Breakdown of Race

tract_merge["Tract Percentage White"] = raceperc(tract_merge["Tract Pop White"], 
                                           tract_merge["Total Tract Pop"])

tract_merge["Tract Percentage Black or African American"] = raceperc(tract_merge["Tract Pop Black or African American"], 
                                                               tract_merge["Total Tract Pop"])

tract_merge["Tract Percentage American Indian and Alaskan Native"] = raceperc(tract_merge["Tract Pop American Indian and Alaska Native"], 
                                                                        tract_merge["Total Tract Pop"])

tract_merge["Tract Percentage Asian"] = raceperc(tract_merge["Tract Pop Asian"], 
                                           tract_merge["Total Tract Pop"])

tract_merge["Tract Percentage Other Race"] = inverseraceperc(tract_merge["Tract Percentage White"], 
                                                       tract_merge["Tract Percentage Black or African American"], 
                                                       tract_merge["Tract Percentage American Indian and Alaskan Native"],
                                                       tract_merge["Tract Percentage Asian"])

In [ ]:
#Create Income String Columns for tooltips to display in Folium

#Cast numeric Income value as String
tract_merge["Tract Income String"] = tract_merge['Tract Per Capita Income - 12 Months'].astype(str)

#Find upper bound of Margin of Error and cast that to String
tract_merge["Tract Income ME Upper"] = upperme(tract_merge["Tract Per Capita Income - 12 Months"],
                                           tract_merge["Tract Per Capita Income ME - 12 Months"])
tract_merge["Tract Income ME Upper String"] = tract_merge['Tract Income ME Upper'].astype(str)

#Find lower bound of Margin of Error and cast that to String
tract_merge["Tract Income ME Lower"] = lowerme(tract_merge["Tract Per Capita Income - 12 Months"],
                                           tract_merge["Tract Per Capita Income ME - 12 Months"])
tract_merge["Tract Income ME Lower String"] = tract_merge['Tract Income ME Lower'].astype(str)

#Fill sentence set to display with Income, Income Margin of Error Range
tract_merge["Tract_Map_Income_Label"] = 'Tract Income Estimate: $' + tract_merge["Tract Income String"] + ' (The US Census Bureau is 90% confident that the true per Capita Income in 2019 was between $' + tract_merge["Tract Income ME Lower String"] + ' and $' + tract_merge["Tract Income ME Upper String"] + ')'

In [ ]:
#Create Pop Density String Columns for tooltips to display in Folium

#Cast numeric Income value as String
tract_merge["Tract Pop Density String"] = tract_merge['Pop Density per sq km'].astype(str)

#Fill sentence set to display with Pop Density
tract_merge["Tract_Map_Pop_Density_Label"] = 'Tract Population Density: ' + tract_merge["Tract Pop Density String"] + ' per sq km'

In [ ]:
#Convert to 4326 Projection, will merge with city specific shapefiles later, and want projection system to match
tract_merge_4326 = tract_merge.to_crs(epsg='4326')

---
## Tract Level: 

##### *Filtering to Houston*
---

In [ ]:
#Merge shapes using inner & intersects methods
#Inner uses intersection of keys from both dfs, but retains only the left geometry 
#intersects returns values that intersect with either the interior or exterior of the larger geometry 
houston_join_tract_4326 = gpd.sjoin(tract_merge_4326,houston_place_4326,how='inner',op='intersects')

In [ ]:
#Check for Nan/Null values
sum(houston_join_tract_4326.isnull().any())

In [ ]:
#Print Nan/Null columns
nan_cols_tract = [i for i in houston_join_tract_4326.columns if houston_join_tract_4326[i].isnull().any()]
print(nan_cols_tract)

In [ ]:
#Print number of values of columns that are Nan/Null
houston_join_tract_4326[houston_join_tract_4326.columns[houston_join_tract_4326.isnull().any()]].isnull().sum()

In [ ]:
houston_tract_nan=houston_join_tract_4326[houston_join_tract_4326.isna().any(axis=1)]
houston_tract_nan

In [ ]:
#Drop Tract with Nan values that is located on the airport
houston_join_tract_4326.drop([3057],inplace=True)

Change remaining Tract with Nan values -- Since the NaN values are in Female Median Age and Female Median Age ME, I assume that there are no Females in the tract. I decide to change the values of each category to the average of those Houston specific columns

In [ ]:
nanreplacement=houston_join_tract_4326["Tract Median Age F"].mean()
nanreplacement

In [ ]:
nanreplacementme=houston_join_tract_4326["Tract Median Age ME F"].mean()
nanreplacementme

In [ ]:
houston_join_tract_4326.at[2029, 'Tract Median Age F'] = nanreplacement

In [ ]:
houston_join_tract_4326.at[2029, 'Tract Median Age ME F'] = nanreplacementme

In [ ]:
#Check that the NaN values are gone
houston_join_tract_4326[houston_join_tract_4326.columns[houston_join_tract_4326.isnull().any()]].isnull().sum()

In [ ]:
#Save File as GeoJSON
houston_join_tract_4326.to_file("houston_join_tract_4326_gjson.geojson", driver='GeoJSON')

In [ ]:
#Create Function to Read GeoJSON Back in
houston_join_tract_4326_gjson = gpd.read_file("houston_join_tract_4326_gjson.geojson", driver='GeoJSON')

In [ ]:
dallas_join_tract_4326.info()

In [ ]:
houston_join_tract_4326[['Pop Density per sq km',
                        'Total Tract Median Age',
                        'Tract Median Age M',
                        'Tract Median Age F',
                        'Tract Per Capita Income - 12 Months',
                        'Tract Percentage White',
                        'Tract Percentage Black or African American',
                        'Tract Percentage Asian']].corr()

In [ ]:
pd.set_option("max_columns", 38)
houston_join_tract_4326.describe(include=[np.number])

In [ ]:
houston_join_describe=houston_join_tract_4326.describe(include=[np.number])

In [ ]:
houston_join_describe=houston_join_describe.drop(columns=['Land Area_left', 
                                                          'Water Area_left', 
                                                          'Tract Perimeter', 
                                                          'Tract Area',
                                                          'State FIPS Code_left',
                                                          'County FIPS Code',
                                                          'Tract FIPS Code', 
                                                          'index_right', 
                                                          'Land Area_right',
                                                          'Water Area_right', 
                                                          'Place Perimeter', 
                                                          'Place Area'])
houston_join_describe

---
## Tract Level: 

##### *Filtering to Dallas*
---

In [ ]:
#Merge shapes using inner & intersects methods
#Inner uses intersection of keys from both dfs, but retains only the left geometry 
#intersects returns values that intersect with either the interior or exterior of the larger geometry 
dallas_join_tract_4326 = gpd.sjoin(tract_merge_4326,dallas_place_4326,how='inner',op='intersects')

In [ ]:
#Check for Nan/Null values
sum(dallas_join_tract_4326.isnull().any())

In [ ]:
#Print Nan/Null columns
nan_cols_tract = [i for i in dallas_join_tract_4326.columns if dallas_join_tract_4326[i].isnull().any()]
print(nan_cols_tract)

In [ ]:
#Print number of values of columns that are Nan/Null
dallas_join_tract_4326[dallas_join_tract_4326.columns[dallas_join_tract_4326.isnull().any()]].isnull().sum()

In [ ]:
dallas_tract_nan=dallas_join_tract_4326[dallas_join_tract_4326.isna().any(axis=1)]
dallas_tract_nan

In [ ]:
#Drop Tract with Nan values since it is located on the airport
dallas_join_tract_4326.drop([1558],inplace=True)

In [ ]:
#Check that the NaN values are gone
dallas_join_tract_4326[dallas_join_tract_4326.columns[dallas_join_tract_4326.isnull().any()]].isnull().sum()

In [ ]:
#Save File as GeoJSON
dallas_join_tract_4326.to_file("dallas_join_tract_4326_gjson.geojson", driver='GeoJSON')

In [ ]:
#Create Function to Read GeoJSON Back in
dallas_join_tract_4326_gjson = gpd.read_file("dallas_join_tract_4326_gjson.geojson", driver='GeoJSON')

---
## Foursquare: Preparing

##### *Writing Functions etc.*
---

In [ ]:
FS_url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&radius={}&limit={}&categoryId={}&oauth_token={}&v={}'
FS_VERSION = 20210213

In [ ]:
gym = '4bf58dd8d48988d175941735' #Gym/Fitness Center 3
park = '4bf58dd8d48988d163941735' #Park 2
FS_LIMIT=50

In [ ]:
#Define function to approximate Radius to use in Foursquare queries
def rapprox(x):
    return ((x / 6.283))
#x will be the perimeter of the tract, this should give a reasonable radius to search
#r = (Perimeter/(2*pi)) 

In [ ]:
#Define function to pull the category name out of the Foursquare results
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [ ]:
#Due to Foursquare API limitations, will have to split Houston tracts list into two parts
def middle_index(list):
    return (len(list)//2)

---
## Foursquare: Gathering Data 

##### *Houston*
---

In [ ]:
#Filter big dataframe to relevant data
FS_houston = houston_join_tract_4326.filter(['Tract Perimeter','Tract Center Lat', 'Tract Center Long'],axis=1)

In [ ]:
#Add Radius column by using rapprox function set up previously
FS_houston["Radius"] = rapprox(FS_houston["Tract Perimeter"])

In [ ]:
#Convert Lon & Lat columns to floats and strip the + sign so they can be put into Foursquare's API
FS_houston['Tract Center Lat'] = (FS_houston['Tract Center Lat'].str.strip('+').astype(float))
FS_houston['Tract Center Long'] = (FS_houston['Tract Center Long'].str.strip('+').astype(float))
FS_houston.Radius = FS_houston.Radius.astype(int)

In [ ]:
#Create blank lists to create list to iterate through for Foursquare API
houston_first_list = []
FS_houston_list = []

In [ ]:
#Write to the list using the filtered dataframe
for index, rows in FS_houston.iterrows(): 
    houston_first_list = [rows['Tract Center Lat'], rows['Tract Center Long'], rows['Radius']]
    FS_houston_list.append(houston_first_list)
print(FS_houston_list)

In [ ]:
#Check to make sure the list is the correct length
print(len(FS_houston_list))

In [ ]:
#Find middle element of list to split into lengths less than 500 (Foursquare API limit is 500)
houston_middle_index=middle_index(FS_houston_list)
print(houston_middle_index)

In [ ]:
#Split list into two parts, first half and second half 
FS_houston_list1=FS_houston_list[:houston_middle_index]
FS_houston_list2=FS_houston_list[houston_middle_index:]

#### *Gym Query*

In [ ]:
#Running loop to get first half of Houston FS gym data
FS_houston_gym_list = []

for v in FS_houston_list1:
    FS_houston_gym_url1 = FS_url.format(
    config.FS_CLIENT_ID, 
    config.FS_CLIENT_SECRET, 
    v[0], 
    v[1], 
    v[2], 
    FS_LIMIT,
    gym, 
    config.FS_ACCESS_TOKEN, 
    FS_VERSION)
    FS_houston_gym_results1 = requests.get(FS_houston_gym_url1).json()
    FS_houston_gym_venues1 = FS_houston_gym_results1['response']['venues']
    FS_houston_gym_list.extend(FS_houston_gym_venues1)

In [ ]:
print(len(FS_houston_gym_list))

In [ ]:
#Running loop to get second half of Houston FS gym data
for v in FS_houston_list2:
    FS_houston_gym_url2 = FS_url.format(
    config.FS_CLIENT_ID, 
    config.FS_CLIENT_SECRET, 
    v[0], 
    v[1], 
    v[2], 
    FS_LIMIT,
    gym, 
    config.FS_ACCESS_TOKEN, 
    FS_VERSION)
    FS_houston_gym_results2 = requests.get(FS_houston_gym_url2).json()
    FS_houston_gym_venues2 = FS_houston_gym_results2['response']['venues']
    FS_houston_gym_list.extend(FS_houston_gym_venues2)

In [ ]:
print(len(FS_houston_gym_list))

In [ ]:
#Normalize json list result so it can be treated as a dataframe
FS_houston_gym_df=json_normalize(FS_houston_gym_list)

In [ ]:
#Clean dataset by dropping duplicate results
FS_houston_gym_df.drop_duplicates(subset='id',inplace=True)

In [ ]:
#Clean dataset by pulling out category names from the list in each row, and only pulling relevant name and location results
filtered_houston_gym_columns = ['name', 'categories'] + [col for col in FS_houston_gym_df.columns if col.startswith('location.')] + ['id']
houston_gym_filtered = FS_houston_gym_df.loc[:, filtered_houston_gym_columns]

houston_gym_filtered['categories'] = houston_gym_filtered.apply(get_category_type, axis=1)

In [ ]:
houston_gym_filtered.head()

In [ ]:
houston_gym_filtered.info()

In [ ]:
#Clean column names
houston_gym_filtered.columns = [column.split('.')[-1] for column in houston_gym_filtered.columns]

In [ ]:
houston_gym_gpd_input=houston_gym_filtered.drop(columns=['labeledLatLngs','formattedAddress'])

In [ ]:
houston_gym_gpd=gpd.GeoDataFrame(houston_gym_gpd_input, geometry=gpd.points_from_xy(houston_gym_gpd_input.lng,
                                                                   houston_gym_gpd_input.lat)
                                )

In [ ]:
#Save File as GeoJSON
houston_gym_gpd.to_file("houston_gym_gpd_gjson.geojson", driver='GeoJSON')

In [ ]:
#Create Function to Read GeoJSON Back in
houston_gym_gpd_gjson = gpd.read_file("houston_gym_gpd_gjson.geojson", driver='GeoJSON')

In [ ]:
#Save File as CSV
houston_gym_filtered.to_csv('houston_gym_filtered_csv.csv')

#### *Park Query*

In [ ]:
#Running loop to get first half of Houston FS park data
FS_houston_park_list = []

for v in FS_houston_list1:
    FS_houston_park_url1 = FS_url.format(
    config.FS_CLIENT_ID, 
    config.FS_CLIENT_SECRET, 
    v[0], 
    v[1], 
    v[2], 
    FS_LIMIT,
    park, 
    config.FS_ACCESS_TOKEN, 
    FS_VERSION)
    FS_houston_park_results1 = requests.get(FS_houston_park_url1).json()
    FS_houston_park_venues1 = FS_houston_park_results1['response']['venues']
    FS_houston_park_list.extend(FS_houston_park_venues1)

In [ ]:
#Running loop to get second half of Houston FS park data
for v in FS_houston_list2:
    FS_houston_park_url2 = FS_url.format(
    config.FS_CLIENT_ID, 
    config.FS_CLIENT_SECRET, 
    v[0], 
    v[1], 
    v[2], 
    FS_LIMIT,
    park, 
    config.FS_ACCESS_TOKEN, 
    FS_VERSION)
    FS_houston_park_results2 = requests.get(FS_houston_park_url2).json()
    FS_houston_park_venues2 = FS_houston_park_results2['response']['venues']
    FS_houston_park_list.extend(FS_houston_park_venues2)

In [ ]:
#Normalize json list result so it can be treated as a dataframe
FS_houston_park_df=json_normalize(FS_houston_park_list)

In [ ]:
#Clean dataset by dropping duplicate results
FS_houston_park_df.drop_duplicates(subset='id',inplace=True)

In [ ]:
#Clean dataset by pulling out category names from the list in each row, and only pulling relevant name and location results
filtered_houston_park_columns = ['name', 'categories'] + [col for col in FS_houston_park_df.columns if col.startswith('location.')] + ['id']
houston_park_filtered = FS_houston_park_df.loc[:, filtered_houston_park_columns]

houston_park_filtered['categories'] = houston_park_filtered.apply(get_category_type, axis=1)

In [ ]:
FS_houston_park_df.info()

In [ ]:
houston_park_filtered.head()

In [ ]:
houston_park_filtered.info()

In [ ]:
#Clean column names
houston_park_filtered.columns = [column.split('.')[-1] for column in houston_park_filtered.columns]

In [ ]:
houston_park_gpd_input=houston_park_filtered.drop(columns=['labeledLatLngs','formattedAddress'])

In [ ]:
houston_park_gpd=gpd.GeoDataFrame(houston_park_gpd_input, geometry=gpd.points_from_xy(houston_park_gpd_input.lng,
                                                                   houston_park_gpd_input.lat)
                                )

In [ ]:
#Save File as GeoJSON
houston_park_gpd.to_file("houston_park_gpd_gjson.geojson", driver='GeoJSON')

In [ ]:
#Create Function to Read GeoJSON Back in
houston_park_gpd_gjson = gpd.read_file("houston_park_gpd_gjson.geojson", driver='GeoJSON')

In [ ]:
#Save File as CSV
houston_park_filtered.to_csv('houston_park_filtered_csv.csv')

---
## Foursquare: Gathering Data 

##### *Dallas*
---

In [ ]:
#Filter big dataframe to relevant data
FS_dallas = dallas_join_tract_4326.filter(['Tract Perimeter','Tract Center Lat', 'Tract Center Long'],axis=1)

In [ ]:
#Add Radius column by using rapprox function set up previously
FS_dallas["Radius"] = rapprox(FS_dallas["Tract Perimeter"])

In [ ]:
#Convert Lon & Lat columns to floats and strip the + sign so they can be put into Foursquare's API
FS_dallas['Tract Center Lat'] = (FS_dallas['Tract Center Lat'].str.strip('+').astype(float))
FS_dallas['Tract Center Long'] = (FS_dallas['Tract Center Long'].str.strip('+').astype(float))
FS_dallas.Radius = FS_dallas.Radius.astype(int)

In [ ]:
#Create blank lists to create list to iterate through for Foursquare API
dallas_first_list = []
FS_dallas_list = []

In [ ]:
#Write to the list using the filtered dataframe
for index, rows in FS_dallas.iterrows(): 
    dallas_first_list = [rows['Tract Center Lat'], rows['Tract Center Long'], rows['Radius']]
    FS_dallas_list.append(dallas_first_list)
print(FS_dallas_list)

In [ ]:
#Check to make sure the list is the correct length
print(len(FS_dallas_list))

#### *Gym Query*

In [ ]:
#Running loop to get Dallas FS gym data
FS_dallas_gym_list = []

for v in FS_dallas_list:
    FS_dallas_gym_url = FS_url.format(
    config.FS_CLIENT_ID, 
    config.FS_CLIENT_SECRET, 
    v[0], 
    v[1], 
    v[2], 
    FS_LIMIT,
    gym, 
    config.FS_ACCESS_TOKEN, 
    FS_VERSION)
    FS_dallas_gym_results = requests.get(FS_dallas_gym_url).json()
    FS_dallas_gym_venues = FS_dallas_gym_results['response']['venues']
    FS_dallas_gym_list.extend(FS_dallas_gym_venues)

In [ ]:
#Normalize json list result so it can be treated as a dataframe
FS_dallas_gym_df=json_normalize(FS_dallas_gym_list)

In [ ]:
FS_dallas_gym_df.info()

In [ ]:
#Clean dataset by dropping duplicate results
FS_dallas_gym_df.drop_duplicates(subset='id',inplace=True)

In [ ]:
#Clean dataset by pulling out category names from the list in each row, and only pulling relevant name and location results
filtered_dallas_gym_columns = ['name', 'categories'] + [col for col in FS_dallas_gym_df.columns if col.startswith('location.')] + ['id']
dallas_gym_filtered = FS_dallas_gym_df.loc[:, filtered_dallas_gym_columns]

dallas_gym_filtered['categories'] = dallas_gym_filtered.apply(get_category_type, axis=1)

In [ ]:
dallas_gym_filtered.head()

In [ ]:
dallas_gym_filtered.info()

In [ ]:
#Clean column names
dallas_gym_filtered.columns = [column.split('.')[-1] for column in dallas_gym_filtered.columns]

In [ ]:
dallas_gym_gpd_input=dallas_gym_filtered.drop(columns=['labeledLatLngs','formattedAddress'])

In [ ]:
dallas_gym_gpd=gpd.GeoDataFrame(dallas_gym_gpd_input, geometry=gpd.points_from_xy(dallas_gym_gpd_input.lng,
                                                                   dallas_gym_gpd_input.lat)
                                )

In [ ]:
#Save File as GeoJSON
dallas_gym_gpd.to_file("dallas_gym_gpd_gjson.geojson", driver='GeoJSON')

In [ ]:
#Create Function to Read GeoJSON Back in
dallas_gym_gpd.to_file("dallas_gym_gpd_gjson.geojson", driver='GeoJSON')
dallas_gym_gpd_gjson = gpd.read_file("dallas_gym_gpd_gjson.geojson", driver='GeoJSON')

In [ ]:
#Save File as CSV
dallas_gym_filtered.to_csv('dallas_gym_filtered_csv.csv')

#### *Park Query*

In [ ]:
#Running loop to get Dallas FS park data
FS_dallas_park_list = []

for v in FS_dallas_list:
    FS_dallas_park_url = FS_url.format(
    config.FS_CLIENT_ID, 
    config.FS_CLIENT_SECRET, 
    v[0], 
    v[1], 
    v[2], 
    FS_LIMIT,
    park, 
    config.FS_ACCESS_TOKEN, 
    FS_VERSION)
    FS_dallas_park_results = requests.get(FS_dallas_park_url).json()
    FS_dallas_park_venues = FS_dallas_park_results['response']['venues']
    FS_dallas_park_list.extend(FS_dallas_park_venues)

In [ ]:
#Normalize json list result so it can be treated as a dataframe
FS_dallas_park_df=json_normalize(FS_dallas_park_list)

In [ ]:
FS_dallas_park_df.info()

In [ ]:
#Clean dataset by dropping duplicate results
FS_dallas_park_df.drop_duplicates(subset='id',inplace=True)

In [ ]:
#Clean dataset by pulling out category names from the list in each row, and only pulling relevant name and location results
filtered_dallas_park_columns = ['name', 'categories'] + [col for col in FS_dallas_park_df.columns if col.startswith('location.')] + ['id']
dallas_park_filtered = FS_dallas_park_df.loc[:, filtered_dallas_park_columns]

dallas_park_filtered['categories'] = dallas_park_filtered.apply(get_category_type, axis=1)

In [ ]:
dallas_park_filtered.head()

In [ ]:
dallas_park_filtered.info()

In [ ]:
#Clean column names
dallas_park_filtered.columns = [column.split('.')[-1] for column in dallas_park_filtered.columns]

In [ ]:
dallas_park_gpd_input=dallas_park_filtered.drop(columns=['labeledLatLngs','formattedAddress'])

In [ ]:
dallas_park_gpd=gpd.GeoDataFrame(dallas_park_gpd_input, geometry=gpd.points_from_xy(dallas_park_gpd_input.lng,
                                                                   dallas_park_gpd_input.lat)
                                )

In [ ]:
#Save File as GeoJSON
dallas_park_gpd.to_file("dallas_park_gpd_gjson.geojson", driver='GeoJSON')

In [ ]:
#Create Function to Read GeoJSON Back in
dallas_park_gpd_gjson = gpd.read_file("dallas_park_gpd_gjson.geojson", driver='GeoJSON')

In [ ]:
#Save File as CSV
dallas_park_filtered.to_csv('dallas_park_filtered_csv.csv')

---
## Foursquare: Adding to Large Datasets

##### *Dallas*
---

In [ ]:
def percap (x,y):
    return (x/y)

In [ ]:
#Add number of gyms and parks to dataframes

In [ ]:
dallas_gym_points_merge = gpd.sjoin(dallas_gym_gpd_gjson,tract_merge_4326)

In [ ]:
dallas_gym_points_merge=dallas_gym_points_merge[['id',
                             'name',
                             'geometry',
                             'GISJOIN']]

In [ ]:
dallas_gym_points_merge.info()

In [ ]:
dallas_gym_counts=(dallas_gym_points_merge.pivot_table(index=['GISJOIN'], aggfunc='size'))

In [ ]:
dallas_gym_counts=dallas_gym_counts.to_frame()

In [ ]:
dallas_gym_counts=dallas_gym_counts.rename({0:'Number of Gyms'}, axis="columns")

In [ ]:
dallas_join_tract_4326_gjson=dallas_join_tract_4326_gjson.rename({'GISJOIN_left':'GISJOIN'}, axis="columns")

In [ ]:
dallas_gym_counts_merge = dallas_join_tract_4326_gjson.merge(dallas_gym_counts, how='left', on='GISJOIN')

In [ ]:
dallas_gym_counts_merge['Number of Gyms'] = dallas_gym_counts_merge['Number of Gyms'].fillna(0)

In [ ]:
dallas_park_points_merge = gpd.sjoin(dallas_park_gpd_gjson,tract_merge_4326)

In [ ]:
dallas_park_points_merge=dallas_park_points_merge[['id',
                             'name',
                             'geometry',
                             'GISJOIN']]

In [ ]:
dallas_park_points_merge.info()

In [ ]:
dallas_park_counts=(dallas_park_points_merge.pivot_table(index=['GISJOIN'], aggfunc='size'))

In [ ]:
dallas_park_counts=dallas_park_counts.to_frame()

In [ ]:
dallas_park_counts=dallas_park_counts.rename({0:'Number of Parks'}, axis="columns")

In [ ]:
dallas_gym_park_counts_merge = dallas_gym_counts_merge.merge(dallas_park_counts, how='left', on='GISJOIN')

In [ ]:
dallas_gym_park_counts_merge['Number of Parks'] = dallas_gym_park_counts_merge['Number of Parks'].fillna(0)

In [ ]:
dallas_gym_park_counts_merge['Gyms per Capita'] = percap(dallas_gym_park_counts_merge['Number of Gyms'],
                                                          dallas_gym_park_counts_merge['Total Tract Pop'])

dallas_gym_park_counts_merge['Parks per Capita'] = percap(dallas_gym_park_counts_merge['Number of Parks'],
                                                          dallas_gym_park_counts_merge['Total Tract Pop'])

In [ ]:
dallas_gym_park_counts_merge['Gyms per sq km'] = popdens(dallas_gym_park_counts_merge['Number of Gyms'],
                                                          dallas_gym_park_counts_merge['Land Area_left'])

dallas_gym_park_counts_merge['Parks per sq km'] = popdens(dallas_gym_park_counts_merge['Number of Parks'],
                                                          dallas_gym_park_counts_merge['Land Area_left'])

In [ ]:
houston_gym_points_merge = gpd.sjoin(houston_gym_gpd_gjson,tract_merge_4326)

In [ ]:
houston_gym_points_merge=houston_gym_points_merge[['id',
                             'name',
                             'geometry',
                             'GISJOIN']]

In [ ]:
houston_gym_points_merge.info()

In [ ]:
houston_gym_counts=(houston_gym_points_merge.pivot_table(index=['GISJOIN'], aggfunc='size'))

In [ ]:
houston_gym_counts=houston_gym_counts.to_frame()

In [ ]:
houston_gym_counts=houston_gym_counts.rename({0:'Number of Gyms'}, axis="columns")

In [ ]:
houston_join_tract_4326_gjson=houston_join_tract_4326_gjson.rename({'GISJOIN_left':'GISJOIN'}, axis="columns")

In [ ]:
houston_gym_counts_merge = houston_join_tract_4326_gjson.merge(houston_gym_counts, how='left', on='GISJOIN')

In [ ]:
houston_gym_counts_merge['Number of Gyms'] = houston_gym_counts_merge['Number of Gyms'].fillna(0)

In [ ]:
houston_park_points_merge = gpd.sjoin(houston_park_gpd_gjson,tract_merge_4326)

In [ ]:
houston_park_points_merge=houston_park_points_merge[['id',
                             'name',
                             'geometry',
                             'GISJOIN']]

In [ ]:
houston_park_points_merge.info()

In [ ]:
houston_park_counts=(houston_park_points_merge.pivot_table(index=['GISJOIN'], aggfunc='size'))

In [ ]:
houston_park_counts=houston_park_counts.to_frame()

In [ ]:
houston_park_counts=houston_park_counts.rename({0:'Number of Parks'}, axis="columns")

In [ ]:
houston_gym_park_counts_merge = houston_gym_counts_merge.merge(houston_park_counts, how='left', on='GISJOIN')

In [ ]:
houston_gym_park_counts_merge['Number of Parks'] = houston_gym_park_counts_merge['Number of Parks'].fillna(0)

In [ ]:
houston_gym_park_counts_merge['Gyms per Capita'] = percap(houston_gym_park_counts_merge['Number of Gyms'],
                                                          houston_gym_park_counts_merge['Total Tract Pop'])

houston_gym_park_counts_merge['Parks per Capita'] = percap(houston_gym_park_counts_merge['Number of Parks'],
                                                          houston_gym_park_counts_merge['Total Tract Pop'])

In [ ]:
houston_gym_park_counts_merge['Gyms per sq km'] = popdens(houston_gym_park_counts_merge['Number of Gyms'],
                                                          houston_gym_park_counts_merge['Land Area_left'])

houston_gym_park_counts_merge['Parks per sq km'] = popdens(houston_gym_park_counts_merge['Number of Parks'],
                                                          houston_gym_park_counts_merge['Land Area_left'])

---
## Looking at Correlation and other Statistical Analyses

##### Houston
---

In [ ]:
from scipy.stats import kendalltau, pearsonr, spearmanr

def kendall_pval(x,y):
    return kendalltau(x,y)[1]
    
def pearsonr_pval(x,y):
    return pearsonr(x,y)[1]
    
def spearmanr_pval(x,y):
    return spearmanr(x,y)[1]

In [ ]:
#Create Z-Score Columns
houston_gym_park_counts_merge['Z Pop Density per sq km']=((houston_gym_park_counts_merge['Pop Density per sq km']-
                                                         houston_gym_park_counts_merge['Pop Density per sq km'].mean())/
                                                         (houston_gym_park_counts_merge['Pop Density per sq km'].std()))

houston_gym_park_counts_merge['Z Total Tract Median Age']=((houston_gym_park_counts_merge['Total Tract Median Age']-
                                                         houston_gym_park_counts_merge['Total Tract Median Age'].mean())/
                                                         (houston_gym_park_counts_merge['Total Tract Median Age'].std()))

houston_gym_park_counts_merge['Z Tract Median Age M']=((houston_gym_park_counts_merge['Tract Median Age M']-
                                                         houston_gym_park_counts_merge['Tract Median Age M'].mean())/
                                                         (houston_gym_park_counts_merge['Tract Median Age M'].std()))

houston_gym_park_counts_merge['Z Tract Median Age F']=((houston_gym_park_counts_merge['Tract Median Age F']-
                                                         houston_gym_park_counts_merge['Tract Median Age F'].mean())/
                                                         (houston_gym_park_counts_merge['Tract Median Age F'].std()))

houston_gym_park_counts_merge['Z Tract Per Capita Income - 12 Months']=((houston_gym_park_counts_merge['Tract Per Capita Income - 12 Months']-
                                                         houston_gym_park_counts_merge['Tract Per Capita Income - 12 Months'].mean())/
                                                         (houston_gym_park_counts_merge['Tract Per Capita Income - 12 Months'].std()))

houston_gym_park_counts_merge['Z Tract Percentage White']=((houston_gym_park_counts_merge['Tract Percentage White']-
                                                         houston_gym_park_counts_merge['Tract Percentage White'].mean())/
                                                         (houston_gym_park_counts_merge['Tract Percentage White'].std()))

houston_gym_park_counts_merge['Z Tract Percentage Black or African American']=((houston_gym_park_counts_merge['Tract Percentage Black or African American']-
                                                         houston_gym_park_counts_merge['Tract Percentage Black or African American'].mean())/
                                                         (houston_gym_park_counts_merge['Tract Percentage Black or African American'].std()))

houston_gym_park_counts_merge['Z Tract Percentage Asian']=((houston_gym_park_counts_merge['Tract Percentage Asian']-
                                                         houston_gym_park_counts_merge['Tract Percentage Asian'].mean())/
                                                         (houston_gym_park_counts_merge['Tract Percentage Asian'].std()))

houston_gym_park_counts_merge['Z Gyms per Capita']=((houston_gym_park_counts_merge['Gyms per Capita']-
                                                         houston_gym_park_counts_merge['Gyms per Capita'].mean())/
                                                         (houston_gym_park_counts_merge['Gyms per Capita'].std()))

houston_gym_park_counts_merge['Z Gyms per sq km']=((houston_gym_park_counts_merge['Gyms per sq km']-
                                                         houston_gym_park_counts_merge['Gyms per sq km'].mean())/
                                                         (houston_gym_park_counts_merge['Gyms per sq km'].std()))

houston_gym_park_counts_merge['Z Parks per Capita']=((houston_gym_park_counts_merge['Parks per Capita']-
                                                         houston_gym_park_counts_merge['Parks per Capita'].mean())/
                                                         (houston_gym_park_counts_merge['Parks per Capita'].std()))

houston_gym_park_counts_merge['Z Parks per sq km']=((houston_gym_park_counts_merge['Parks per sq km']-
                                                         houston_gym_park_counts_merge['Parks per sq km'].mean())/
                                                         (houston_gym_park_counts_merge['Parks per sq km'].std()))

In [ ]:
houston_corr_coeff=houston_gym_park_counts_merge[['Pop Density per sq km',
                                                  'Total Tract Median Age',
                                                  'Tract Per Capita Income - 12 Months',
                                                  'Tract Percentage White',
                                                  'Number of Gyms',
                                                  'Number of Parks']].corr(method=pearsonr_pval)
houston_corr_coeff

In [ ]:
fig, ax = plt.subplots()
im = ax.pcolor(houston_corr_coeff, cmap='RdBu', vmin=-1, vmax=1)

#label names
row_labels = houston_corr_coeff.columns
col_labels = houston_corr_coeff.index

#move ticks and labels to the center
ax.set_xticks(np.arange(len(houston_corr_coeff)))
ax.set_yticks(np.arange(len(houston_corr_coeff)))

#insert labels
ax.set_xticklabels(row_labels)
ax.set_yticklabels(col_labels)

#rotate label if too long
plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")

#set title
plt.title(label="Houston Correlation Heatmap")

fig.colorbar(im)
plt.show()
#fig.savefig("HoustonHeat.png")

In [ ]:
z_houston_corr_coeff=houston_gym_park_counts_merge[['Z Pop Density per sq km',
                               'Z Total Tract Median Age',
                               'Z Tract Median Age M',
                               'Z Tract Median Age F',
                               'Z Tract Per Capita Income - 12 Months',
                               'Z Tract Percentage White',
                               'Z Tract Percentage Black or African American',
                               'Z Tract Percentage Asian',
                               'Z Gyms per Capita',
                               'Z Gyms per sq km',
                               'Z Parks per Capita',
                               'Z Parks per sq km']].corr()
z_houston_corr_coeff

In [ ]:
houston_p_values=houston_gym_park_counts_merge[['Pop Density per sq km',
                               'Total Tract Median Age',
                               'Tract Median Age M',
                               'Tract Median Age F',
                               'Tract Per Capita Income - 12 Months',
                               'Tract Percentage White',
                               'Tract Percentage Black or African American',
                               'Tract Percentage Asian',
                               'Gyms per Capita',
                               'Gyms per sq km',
                               'Parks per Capita',
                               'Parks per sq km']].corr(method=pearsonr_pval)
houston_p_values

In [ ]:
houston_describe=houston_gym_park_counts_merge.describe(include=[np.number])
houston_describe

In [ ]:
#Save Final Houston GeoDataframe to GJSON and create function to read it back in
houston_gym_park_counts_merge.to_file("houston_gym_park_counts_merge_gjson.geojson", driver='GeoJSON')
houston_gym_park_counts_merge_gjson = gpd.read_file("houston_gym_park_counts_merge_gjson.geojson", driver='GeoJSON')

---
## Looking at Correlation and other Statistical Analyses

##### Dallas
---

In [ ]:
#Create Z-Score Columns
dallas_gym_park_counts_merge['Z Pop Density per sq km']=((dallas_gym_park_counts_merge['Pop Density per sq km']-
                                                         dallas_gym_park_counts_merge['Pop Density per sq km'].mean())/
                                                         (dallas_gym_park_counts_merge['Pop Density per sq km'].std()))

dallas_gym_park_counts_merge['Z Total Tract Median Age']=((dallas_gym_park_counts_merge['Total Tract Median Age']-
                                                         dallas_gym_park_counts_merge['Total Tract Median Age'].mean())/
                                                         (dallas_gym_park_counts_merge['Total Tract Median Age'].std()))

dallas_gym_park_counts_merge['Z Tract Median Age M']=((dallas_gym_park_counts_merge['Tract Median Age M']-
                                                         dallas_gym_park_counts_merge['Tract Median Age M'].mean())/
                                                         (dallas_gym_park_counts_merge['Tract Median Age M'].std()))

dallas_gym_park_counts_merge['Z Tract Median Age F']=((dallas_gym_park_counts_merge['Tract Median Age F']-
                                                         dallas_gym_park_counts_merge['Tract Median Age F'].mean())/
                                                         (dallas_gym_park_counts_merge['Tract Median Age F'].std()))

dallas_gym_park_counts_merge['Z Tract Per Capita Income - 12 Months']=((dallas_gym_park_counts_merge['Tract Per Capita Income - 12 Months']-
                                                         dallas_gym_park_counts_merge['Tract Per Capita Income - 12 Months'].mean())/
                                                         (dallas_gym_park_counts_merge['Tract Per Capita Income - 12 Months'].std()))

dallas_gym_park_counts_merge['Z Tract Percentage White']=((dallas_gym_park_counts_merge['Tract Percentage White']-
                                                         dallas_gym_park_counts_merge['Tract Percentage White'].mean())/
                                                         (dallas_gym_park_counts_merge['Tract Percentage White'].std()))

dallas_gym_park_counts_merge['Z Tract Percentage Black or African American']=((dallas_gym_park_counts_merge['Tract Percentage Black or African American']-
                                                         dallas_gym_park_counts_merge['Tract Percentage Black or African American'].mean())/
                                                         (dallas_gym_park_counts_merge['Tract Percentage Black or African American'].std()))

dallas_gym_park_counts_merge['Z Tract Percentage Asian']=((dallas_gym_park_counts_merge['Tract Percentage Asian']-
                                                         dallas_gym_park_counts_merge['Tract Percentage Asian'].mean())/
                                                         (dallas_gym_park_counts_merge['Tract Percentage Asian'].std()))

dallas_gym_park_counts_merge['Z Gyms per Capita']=((dallas_gym_park_counts_merge['Gyms per Capita']-
                                                         dallas_gym_park_counts_merge['Gyms per Capita'].mean())/
                                                         (dallas_gym_park_counts_merge['Gyms per Capita'].std()))

dallas_gym_park_counts_merge['Z Gyms per sq km']=((dallas_gym_park_counts_merge['Gyms per sq km']-
                                                         dallas_gym_park_counts_merge['Gyms per sq km'].mean())/
                                                         (dallas_gym_park_counts_merge['Gyms per sq km'].std()))

dallas_gym_park_counts_merge['Z Parks per Capita']=((dallas_gym_park_counts_merge['Parks per Capita']-
                                                         dallas_gym_park_counts_merge['Parks per Capita'].mean())/
                                                         (dallas_gym_park_counts_merge['Parks per Capita'].std()))

dallas_gym_park_counts_merge['Z Parks per sq km']=((dallas_gym_park_counts_merge['Parks per sq km']-
                                                         dallas_gym_park_counts_merge['Parks per sq km'].mean())/
                                                         (dallas_gym_park_counts_merge['Parks per sq km'].std()))

dallas_gym_park_counts_merge['Z Total Tract Pop']=((dallas_gym_park_counts_merge['Total Tract Pop']-
                                                         dallas_gym_park_counts_merge['Total Tract Pop'].mean())/
                                                         (dallas_gym_park_counts_merge['Total Tract Pop'].std()))

dallas_gym_park_counts_merge['Z Number of Gyms']=((dallas_gym_park_counts_merge['Number of Gyms']-
                                                         dallas_gym_park_counts_merge['Number of Gyms'].mean())/
                                                         (dallas_gym_park_counts_merge['Number of Gyms'].std()))

dallas_gym_park_counts_merge['Z Number of Parks']=((dallas_gym_park_counts_merge['Number of Parks']-
                                                         dallas_gym_park_counts_merge['Number of Parks'].mean())/
                                                         (dallas_gym_park_counts_merge['Number of Parks'].std()))

dallas_gym_park_counts_merge['Z Land Area']=((dallas_gym_park_counts_merge['Land Area_left']-
                                                         dallas_gym_park_counts_merge['Land Area_left'].mean())/
                                                         (dallas_gym_park_counts_merge['Land Area_left'].std()))

In [ ]:
z_dallas_corr_coeff=dallas_gym_park_counts_merge[['Z Total Tract Pop',
                                                  'Z Total Tract Median Age',
                                                  'Z Tract Median Age M',
                                                  'Z Tract Median Age F',
                                                  'Z Tract Per Capita Income - 12 Months',
                                                  'Z Tract Percentage White',
                                                  'Z Tract Percentage Black or African American',
                                                  'Z Tract Percentage Asian',
                                                  'Z Number of Gyms',
                                                  'Z Number of Parks',
                                                  'Z Land Area']].corr()
z_dallas_corr_coeff

In [ ]:
plt.figure(figsize=(15, 10))

sns.set(font_scale=1.5)
sns.set_style('whitegrid')

ax=sns.regplot(x='Z Tract Per Capita Income - 12 Months', 
               y='Z Total Tract Median Age', 
               data=dallas_gym_park_counts_merge,
               color='green',
               marker='+',
               scatter_kws={'s': 200})

ax.set(xlabel='Per Capita Income (Z-Score)', 
       ylabel='Median Age (Z-Score)')

ax.set(xlim=(-1,6),
       ylim=(-3,5))

ax.set(title='Regression - Median Age vs. Per Capita Income (Normalized as Z-Scores)')

In [ ]:
sns.displot(dallas_gym_park_counts_merge, x="Tract Per Capita Income - 12 Months", kind="kde")

In [ ]:
sns.displot(houston_gym_park_counts_merge, x="Tract Per Capita Income - 12 Months", kind="kde")

In [ ]:
fig, ax = plt.subplots()
z=dallas_gym_park_counts_merge['Pop Density per sq km']
q=houston_gym_park_counts_merge['Pop Density per sq km']
for a in [z, q]:
    sns.distplot(a, kde=True)
ax.set_xlim([0, 20000])
fig.legend(labels=['Dallas','Houston'])
fig.savefig("Pop.png")

In [ ]:
fig, ax = plt.subplots()
z=dallas_gym_park_counts_merge['Tract Per Capita Income - 12 Months']
q=houston_gym_park_counts_merge['Tract Per Capita Income - 12 Months']
for a in [z, q]:
    sns.distplot(a, kde=True)
ax.set_xlim([0, 200000])
fig.legend(labels=['Dallas','Houston'])
fig.savefig("Income.png")

In [ ]:
fig, ax = plt.subplots()
z=dallas_gym_park_counts_merge['Total Tract Median Age']
q=houston_gym_park_counts_merge['Total Tract Median Age']
for a in [z, q]:
    sns.distplot(a, kde=True)
#ax.set_xlim([0, 200000])
fig.legend(labels=['Dallas','Houston'])
fig.savefig("Age.png")

In [ ]:
fig, ax = plt.subplots()
z=dallas_gym_park_counts_merge['Tract Pop White']
q=houston_gym_park_counts_merge['Tract Pop White']
for a in [z, q]:
    sns.distplot(a, kde=True)
#ax.set_xlim([0, 100])
fig.legend(labels=['Dallas','Houston'])
fig.savefig("Whitepop.png")

In [ ]:
fig, ax = plt.subplots()
z=dallas_gym_park_counts_merge['Tract Pop Black or African American']
q=houston_gym_park_counts_merge['Tract Pop Black or African American']
for a in [z, q]:
    sns.distplot(a, kde=True)
#ax.set_xlim([0, 100])
fig.legend(labels=['Dallas','Houston'])
fig.savefig("Blackpop.png")

In [ ]:
fig, ax = plt.subplots()
z=dallas_gym_park_counts_merge['Tract Pop Asian']
q=houston_gym_park_counts_merge['Tract Pop Asian']
for a in [z, q]:
    sns.distplot(a, kde=True)
#ax.set_xlim([0, 100])
fig.legend(labels=['Dallas','Houston'])
fig.savefig("Asianpop.png")

In [ ]:
fig, ax = plt.subplots()
z=dallas_gym_park_counts_merge['Tract Percentage Other Race']
q=houston_gym_park_counts_merge['Tract Percentage Other Race']
for a in [z, q]:
    sns.distplot(a, kde=True)
#ax.set_xlim([0, 100])
fig.legend(labels=['Dallas','Houston'])
#fig.savefig("OtherRace.png")

In [ ]:
plt.figure(figsize=(15, 10))

#slope, intercept, r_value, p_value, std_err = stats.linregress(tips['total_bill'],tips['tip'])

sns.set(font_scale=1.5)
sns.set_style('whitegrid')

ax=sns.regplot(x='Z Gyms per Capita', 
               y='Z Tract Per Capita Income - 12 Months', 
               data=dallas_gym_park_counts_merge,
               color='green',
               marker='+',
               scatter_kws={'s': 200})

ax.set(xlabel='Per Capita Gyms (Z-Score)', 
       ylabel='Per Capita Income (Z-Score)')

#ax.set(xlim=(-1,4),
#       ylim=(-2,5))

ax.set(title='Regression - Per Capita Income vs. Per Capita Gyms (Normalized with Z-Scores)')

In [ ]:
for w, b, a, i, o in dallas_gym_park_counts_merge

In [ ]:
dallas_corr_coeff=dallas_gym_park_counts_merge[['Pop Density per sq km',
                               'Total Tract Median Age',
                               'Tract Median Age M',
                               'Tract Median Age F',
                               'Tract Per Capita Income - 12 Months',
                               'Tract Percentage White',
                               'Tract Percentage Black or African American',
                               'Tract Percentage Asian',
                               'Gyms per Capita',
                               'Gyms per sq km',
                               'Parks per Capita',
                               'Parks per sq km']].corr()
dallas_corr_coeff

In [ ]:
dallas_describe=dallas_gym_park_counts_merge.describe(include=[np.number])
dallas_describe

In [ ]:
#Save Final Dallas GeoDataframe to GJSON and create function to read it back in
dallas_gym_park_counts_merge.to_file("dallas_gym_park_counts_merge_gjson.geojson", driver='GeoJSON')
dallas_gym_park_counts_merge_gjson = gpd.read_file("dallas_gym_park_counts_merge_gjson.geojson", driver='GeoJSON')

---
## Maps: Set Up 

##### *Houston*
---

In [ ]:
#Look at Quantile values of Income to find a good range for color scale
houston_min_income, houston_max_income = houston_join_tract_4326['Tract Per Capita Income - 12 Months'].quantile([0,.95]).apply(lambda x: round(x, 2))
houston_mean_income = round(houston_join_tract_4326['Tract Per Capita Income - 12 Months'].mean(),2)
print(f"Min: {houston_min_income}", f"Max: {houston_max_income}", f"Mean: {houston_mean_income}", sep="\n\n")

In [ ]:
#Set Income Scale
houstontractincomecolors = linear.Greens_05.scale(0,
                                                   100000)
houstontractincomecolors.caption = 'Tract Per Capita Income - Past 12 Months'

In [ ]:
#Look at Quantile values of Population Density to find a good range for color scale
houston_min_popdense, houston_max_popdense = houston_join_tract_4326['Pop Density per sq km'].quantile([0.01,.95]).apply(lambda x: round(x, 2))
houston_mean_popdense = round(houston_join_tract_4326['Pop Density per sq km'].mean(),2)
print(f"Min: {houston_min_popdense}", f"Max: {houston_max_popdense}", f"Mean: {houston_mean_popdense}", sep="\n\n")

In [ ]:
#Set Population Density Scale
houstontractpopulationcolors = linear.Reds_05.scale(0,
                                                   5000)
houstontractpopulationcolors.caption = 'Tract Population Density (/sq km)'

---
## Maps: Generation 

##### *Houston*
---

##### Houston Income

In [ ]:
#Create map centered on Houston
final_houston_map = folium.Map(location=[29.85027,-95.37473],
                   tiles=None, height=700, width=1100, zoom_start=10)

#Clean Popup Layer
houston_tract_popup_layer=folium.GeoJson(houston_gym_park_counts_merge_gjson, 
       name = 'Simple Tract Outlines',
       style_function = lambda feature: {
           'color': 'black',
           'weight': .35, 
           'lineOpacity': 1,
           'fillColor': 'lightgrey',
           'fillOpacity': .8}, 
       highlight_function=lambda x: {
            'color': 'black',
            'weight': 2,
            'lineOpacity': .8,
            'fillColor': 'grey',
            'fillOpacity': .5},
       tooltip = folium.GeoJsonTooltip(fields=('Tract Area Name', 
                                               'Tract Per Capita Income - 12 Months',
                                               'Pop Density per sq km',
                                               'Number of Parks',
                                               'Number of Gyms',
                                               'Tract Percentage White',
                                               'Tract Percentage Black or African American',
                                               'Tract Percentage Asian',
                                               'Tract Percentage American Indian and Alaskan Native',
                                               'Tract Percentage Other Race'),
                                       aliases=('Census Tract (#, County Name, State):',
                                                'Tract Income (2019 Inflation Adjusted USD):',
                                                'Tract Population Density (/sq km):',
                                                'Number of Parks:',
                                                'Number of Gyms:',
                                                'Population % White:',
                                                'Population % Black/African American:',
                                                'Population % Asian:',
                                                'Population % American Indian',
                                                'Population % Other'),
                                       sticky=True,
                                       localize=True),
       show = True,
       overlay = False,
       zoom_on_click = True)
folium.TileLayer(tiles='https://server.arcgisonline.com/ArcGIS/rest/services/Canvas/World_Light_Gray_Base/MapServer/tile/{z}/{y}/{x}', 
                 attr='Tiles &copy; Esri &mdash; Esri, DeLorme, NAVTEQ', 
                 name='Tract Base').add_to(houston_tract_popup_layer)
houston_tract_popup_layer.add_to(final_houston_map)


#Tract Income & Population Density Combined
houston_pop_income_layer=folium.GeoJson(houston_gym_park_counts_merge_gjson, 
       name='Per Capita Income & Population Density', 
       style_function = lambda feature: {
           'color': 'black',
           'weight': 0, 
           'fillColor': houstontractpopulationcolors(feature['properties']['Pop Density per sq km']), 
           'fillOpacity': .5,
           'highlight': True}, 
       overlay = False,
       show = True
        )
folium.GeoJson(houston_gym_park_counts_merge_gjson, 
       style_function = lambda feature: {
           'color': 'black',
           'weight': 0, 
           'fillColor': houstontractincomecolors(feature['properties']['Tract Per Capita Income - 12 Months']), 
           'fillOpacity': .5,
           'highlight': True}, 
       overlay = False
        ).add_to(houston_pop_income_layer)
folium.TileLayer(tiles='https://server.arcgisonline.com/ArcGIS/rest/services/Canvas/World_Light_Gray_Base/MapServer/tile/{z}/{y}/{x}', 
                 attr='Tiles &copy; Esri &mdash; Esri, DeLorme, NAVTEQ', 
                 name='Tract Base').add_to(houston_pop_income_layer)
houston_tract_popup_layer1=folium.GeoJson(houston_gym_park_counts_merge_gjson, 
       style_function = lambda feature: {
           'color': 'black',
           'weight': .35, 
           'fillColor': 'white',
           'fillOpacity': 0}, 
       highlight_function=lambda x: {
            'color': 'black',
            'weight': 2,
            'lineOpacity': .8,
            'fillColor': 'grey',
            'fillOpacity': .25},
       tooltip = folium.GeoJsonTooltip(fields=('Tract Area Name', 
                                               'Tract Per Capita Income - 12 Months',
                                               'Pop Density per sq km',
                                               'Number of Parks',
                                               'Number of Gyms',
                                               'Tract Percentage White',
                                               'Tract Percentage Black or African American',
                                               'Tract Percentage Asian',
                                               'Tract Percentage American Indian and Alaskan Native',
                                               'Tract Percentage Other Race'),
                                       aliases=('Census Tract (#, County Name, State):',
                                                'Tract Income (2019 Inflation Adjusted USD):',
                                                'Tract Population Density (/sq km):',
                                                'Number of Parks:',
                                                'Number of Gyms:',
                                                'Population % White:',
                                                'Population % Black/African American:',
                                                'Population % Asian:',
                                                'Population % American Indian',
                                                'Population % Other'),
                                       sticky=True,
                                       localize=True),
       show = True,
       overlay = False,
       zoom_on_click = True).add_to(houston_pop_income_layer)
houston_pop_income_layer.add_to(final_houston_map)


#Tract Population Density Color Scale Layer
houston_population_layer=folium.GeoJson(houston_gym_park_counts_merge_gjson, 
       name='Population Density', 
       style_function = lambda feature: {
           'color': 'black',
           'weight': 0, 
           'fillColor': houstontractpopulationcolors(feature['properties']['Pop Density per sq km']), 
           'fillOpacity': .5,
           'highlight': True}, 
       overlay = False
        )
folium.TileLayer(tiles='https://server.arcgisonline.com/ArcGIS/rest/services/Canvas/World_Light_Gray_Base/MapServer/tile/{z}/{y}/{x}', 
                 attr='Tiles &copy; Esri &mdash; Esri, DeLorme, NAVTEQ', 
                 name='Tract Base').add_to(houston_population_layer)
houston_tract_popup_layer2=folium.GeoJson(houston_gym_park_counts_merge_gjson, 
       style_function = lambda feature: {
           'color': 'black',
           'weight': .35, 
           'fillColor': 'white',
           'fillOpacity': 0}, 
       highlight_function=lambda x: {
            'color': 'black',
            'weight': 2,
            'lineOpacity': .8,
            'fillColor': 'grey',
            'fillOpacity': .25},
       tooltip = folium.GeoJsonTooltip(fields=('Tract Area Name', 
                                               'Tract Per Capita Income - 12 Months',
                                               'Pop Density per sq km',
                                               'Number of Parks',
                                               'Number of Gyms',
                                               'Tract Percentage White',
                                               'Tract Percentage Black or African American',
                                               'Tract Percentage Asian',
                                               'Tract Percentage American Indian and Alaskan Native',
                                               'Tract Percentage Other Race'),
                                       aliases=('Census Tract (#, County Name, State):',
                                                'Tract Income (2019 Inflation Adjusted USD):',
                                                'Tract Population Density (/sq km):',
                                                'Number of Parks:',
                                                'Number of Gyms:',
                                                'Population % White:',
                                                'Population % Black/African American:',
                                                'Population % Asian:',
                                                'Population % American Indian',
                                                'Population % Other'),
                                       sticky=True,
                                       localize=True),
       show = True,
       overlay = False,
       zoom_on_click = True).add_to(houston_population_layer)
houston_population_layer.add_to(final_houston_map)


#Tract Income Color Scale Layer
houston_income_layer=folium.GeoJson(houston_gym_park_counts_merge_gjson, 
       name='Per Capita Income (2019)', 
       style_function = lambda feature: {
           'color': 'black',
           'weight': 0, 
           'fillColor': houstontractincomecolors(feature['properties']['Tract Per Capita Income - 12 Months']), 
           'fillOpacity': .5,
           'highlight': True}, 
       overlay = False
        )
folium.TileLayer(tiles='https://server.arcgisonline.com/ArcGIS/rest/services/Canvas/World_Light_Gray_Base/MapServer/tile/{z}/{y}/{x}', 
                 attr='Tiles &copy; Esri &mdash; Esri, DeLorme, NAVTEQ', 
                 name='Tract Base').add_to(houston_income_layer)
houston_tract_popup_layer3=folium.GeoJson(houston_gym_park_counts_merge_gjson,  
       style_function = lambda feature: {
           'color': 'black',
           'weight': .35, 
           'fillColor': 'white',
           'fillOpacity': 0}, 
       highlight_function=lambda x: {
            'color': 'black',
            'weight': 2,
            'lineOpacity': .8,
            'fillColor': 'grey',
            'fillOpacity': .25},
       tooltip = folium.GeoJsonTooltip(fields=('Tract Area Name', 
                                               'Tract Per Capita Income - 12 Months',
                                               'Pop Density per sq km',
                                               'Number of Parks',
                                               'Number of Gyms',
                                               'Tract Percentage White',
                                               'Tract Percentage Black or African American',
                                               'Tract Percentage Asian',
                                               'Tract Percentage American Indian and Alaskan Native',
                                               'Tract Percentage Other Race'),
                                       aliases=('Census Tract (#, County Name, State):',
                                                'Tract Income (2019 Inflation Adjusted USD):',
                                                'Tract Population Density (/sq km):',
                                                'Number of Parks:',
                                                'Number of Gyms:',
                                                'Population % White:',
                                                'Population % Black/African American:',
                                                'Population % Asian:',
                                                'Population % American Indian',
                                                'Population % Other'),
                                       sticky=True,
                                       localize=True),
       show = True,
       overlay = False,
       zoom_on_click = True).add_to(houston_income_layer)
houston_income_layer.add_to(final_houston_map)


#County Outline Layer
houston_county_outline_layer=folium.GeoJson(houston_join_county_4326_gjson, 
       name='Surrounding Counties (Purple Outlines)', 
       style_function = lambda feature: {
           'color': 'purple',
           'weight': 2, 
           'fillOpacity': 0,
           'interactive': False
       },
       show=False
       ).add_to(final_houston_map)

#City Outline Layer
houston_city_outline_layer=folium.GeoJson(houston_join_place_4326_gjson, 
       name='Houston City Limits (Blue Outline)', 
       style_function = lambda feature: {
           'color': 'blue',
           'weight': 2, 
           'fillOpacity': 0,
           'interactive': False
       },
       show=False
       ).add_to(final_houston_map)

#Add Foursquare gym results as markers
houston_gym_markers = folium.FeatureGroup(name='Gym Locations (Yellow Dots)', overlay=True, control=True, show=False)
for lat, lng, label in zip(houston_gym_filtered.lat, houston_gym_filtered.lng, houston_gym_filtered.name):
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        color='gray',
        weight=1,
        popup=label,
        fill = True,
        fill_color='yellow',
        fill_opacity=0.8
    ).add_to(houston_gym_markers)
houston_gym_markers.add_to(final_houston_map)

#Add Foursquare gym results as markers
houston_park_markers = folium.FeatureGroup(name='Park Locations (Blue Dots)', overlay=True, control=True, show=False)
for lat, lng, label in zip(houston_park_filtered.lat, houston_park_filtered.lng, houston_park_filtered.name):
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        color='gray',
        weight=1,
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.8,
    ).add_to(houston_park_markers)
houston_park_markers.add_to(final_houston_map)


#Add Color Scales
houstontractincomecolors.add_to(final_houston_map)
houstontractpopulationcolors.add_to(final_houston_map)


#Set order of layers
final_houston_map.keep_in_front(houston_tract_popup_layer,
                               houston_population_layer,
                               houston_income_layer,
                               houston_pop_income_layer,
                               houston_city_outline_layer,
                               houston_county_outline_layer,
                               houston_gym_markers,
                               houston_park_markers
                               )

#Add layer control
folium.LayerControl(position = 'topright',
                    collapsed = False).add_to(final_houston_map)

final_houston_map

In [ ]:
final_houston_map.save('Houston_Map.html')

In [ ]:
#Create map centered on Houston
houston_park_cluster_map = folium.Map(location=[29.85027,-95.37473],
                   tiles=None, height=700, width=1100, zoom_start=10)

#Clean Tract Outline Layer
houston_tract_popup_layer=folium.GeoJson(houston_gym_park_counts_merge_gjson, 
       name = 'Simple Tract Outlines',
       style_function = lambda feature: {
           'color': 'black',
           'weight': .35, 
           'lineOpacity': 1,
           'fillColor': 'lightgrey',
           'fillOpacity': .8}, 
       show = True,
       overlay = False)
folium.TileLayer(tiles='https://server.arcgisonline.com/ArcGIS/rest/services/Canvas/World_Light_Gray_Base/MapServer/tile/{z}/{y}/{x}', 
                 attr='Tiles &copy; Esri &mdash; Esri, DeLorme, NAVTEQ', 
                 name='Tract Base').add_to(houston_tract_popup_layer)
houston_park_cluster1 = plugins.MarkerCluster(name='Park Clusters',
                                            options={'showCoverageOnHover': True,
                                                     'zoomToBoundsOnClick': True,
                                                     'disableClusteringAtZoom': 15,
                                                     'removeOutsideVisibleBounds': True,
                                                     'spiderfyOnMaxZoom': False},
                                            show=False)
for lat, lng, label in zip(houston_park_filtered.lat, houston_park_filtered.lng, houston_park_filtered.name):
    folium.Marker(
        location=[lat, lng],
        icon=folium.Icon(color='green', icon_color='white', icon='tree', prefix='fa'),
        popup=label,
    ).add_to(houston_park_cluster1)
houston_park_cluster1.add_to(houston_tract_popup_layer)
houston_tract_popup_layer.add_to(houston_park_cluster_map)

#Tract Income & Population Density Combined
houston_pop_income_layer=folium.GeoJson(houston_gym_park_counts_merge_gjson, 
       name='Per Capita Income & Population Density', 
       style_function = lambda feature: {
           'color': 'black',
           'weight': 0, 
           'fillColor': houstontractpopulationcolors(feature['properties']['Pop Density per sq km']), 
           'fillOpacity': .5,
           'highlight': True}, 
       overlay = False,
       show = True
        )
folium.GeoJson(houston_gym_park_counts_merge_gjson, 
       style_function = lambda feature: {
           'color': 'black',
           'weight': .35, 
           'fillColor': houstontractincomecolors(feature['properties']['Tract Per Capita Income - 12 Months']), 
           'fillOpacity': .5,
           'highlight': True}, 
       overlay = False
        ).add_to(houston_pop_income_layer)
folium.TileLayer(tiles='https://server.arcgisonline.com/ArcGIS/rest/services/Canvas/World_Light_Gray_Base/MapServer/tile/{z}/{y}/{x}', 
                 attr='Tiles &copy; Esri &mdash; Esri, DeLorme, NAVTEQ', 
                 name='Tract Base').add_to(houston_pop_income_layer)
houston_park_cluster2 = plugins.MarkerCluster(name='Park Clusters',
                                            options={'showCoverageOnHover': True,
                                                     'zoomToBoundsOnClick': True,
                                                     'disableClusteringAtZoom': 15,
                                                     'removeOutsideVisibleBounds': True,
                                                     'spiderfyOnMaxZoom': False},
                                            show=False)
for lat, lng, label in zip(houston_park_filtered.lat, houston_park_filtered.lng, houston_park_filtered.name):
    folium.Marker(
        location=[lat, lng],
        icon=folium.Icon(color='green', icon_color='white', icon='tree', prefix='fa'),
        popup=label,
    ).add_to(houston_park_cluster2)
houston_park_cluster2.add_to(houston_pop_income_layer)
houston_pop_income_layer.add_to(houston_park_cluster_map)


#Tract Population Density Color Scale Layer
houston_population_layer=folium.GeoJson(houston_gym_park_counts_merge_gjson, 
       name='Population Density', 
       style_function = lambda feature: {
           'color': 'black',
           'weight': .35, 
           'fillColor': houstontractpopulationcolors(feature['properties']['Pop Density per sq km']), 
           'fillOpacity': .5,
           'highlight': True}, 
       overlay = False
        )
folium.TileLayer(tiles='https://server.arcgisonline.com/ArcGIS/rest/services/Canvas/World_Light_Gray_Base/MapServer/tile/{z}/{y}/{x}', 
                 attr='Tiles &copy; Esri &mdash; Esri, DeLorme, NAVTEQ', 
                 name='Tract Base').add_to(houston_population_layer)
houston_park_cluster3 = plugins.MarkerCluster(name='Park Clusters',
                                            options={'showCoverageOnHover': True,
                                                     'zoomToBoundsOnClick': True,
                                                     'disableClusteringAtZoom': 15,
                                                     'removeOutsideVisibleBounds': True,
                                                     'spiderfyOnMaxZoom': False},
                                            show=False)
for lat, lng, label in zip(houston_park_filtered.lat, houston_park_filtered.lng, houston_park_filtered.name):
    folium.Marker(
        location=[lat, lng],
        icon=folium.Icon(color='green', icon_color='white', icon='tree', prefix='fa'),
        popup=label,
    ).add_to(houston_park_cluster3)
houston_park_cluster3.add_to(houston_population_layer)
houston_population_layer.add_to(houston_park_cluster_map)


#Tract Income Color Scale Layer
houston_income_layer=folium.GeoJson(houston_gym_park_counts_merge_gjson, 
       name='Per Capita Income (2019)', 
       style_function = lambda feature: {
           'color': 'black',
           'weight': .35, 
           'fillColor': houstontractincomecolors(feature['properties']['Tract Per Capita Income - 12 Months']), 
           'fillOpacity': .5,
           'highlight': True}, 
       overlay = False
        )
folium.TileLayer(tiles='https://server.arcgisonline.com/ArcGIS/rest/services/Canvas/World_Light_Gray_Base/MapServer/tile/{z}/{y}/{x}', 
                 attr='Tiles &copy; Esri &mdash; Esri, DeLorme, NAVTEQ', 
                 name='Tract Base').add_to(houston_income_layer)
houston_park_cluster4 = plugins.MarkerCluster(name='Park Clusters',
                                            options={'showCoverageOnHover': True,
                                                     'zoomToBoundsOnClick': True,
                                                     'disableClusteringAtZoom': 15,
                                                     'removeOutsideVisibleBounds': True,
                                                     'spiderfyOnMaxZoom': False},
                                            show=False)
for lat, lng, label in zip(houston_park_filtered.lat, houston_park_filtered.lng, houston_park_filtered.name):
    folium.Marker(
        location=[lat, lng],
        icon=folium.Icon(color='green', icon_color='white', icon='tree', prefix='fa'),
        popup=label,
    ).add_to(houston_park_cluster4)
houston_park_cluster4.add_to(houston_income_layer)
houston_income_layer.add_to(houston_park_cluster_map)


#County Outline Layer
houston_county_outline_layer=folium.GeoJson(houston_join_county_4326_gjson, 
       name='Surrounding Counties (Purple Outlines)', 
       style_function = lambda feature: {
           'color': 'purple',
           'weight': 2, 
           'fillOpacity': 0,
           'interactive': False
       },
       show=False
       ).add_to(houston_park_cluster_map)

#City Outline Layer
houston_city_outline_layer=folium.GeoJson(houston_join_place_4326_gjson, 
       name='Houston City Limits (Blue Outline)', 
       style_function = lambda feature: {
           'color': 'blue',
           'weight': 2, 
           'fillOpacity': 0,
           'interactive': False
       },
       show=False
       ).add_to(houston_park_cluster_map)


#Add Color Scales
houstontractincomecolors.add_to(houston_park_cluster_map)
houstontractpopulationcolors.add_to(houston_park_cluster_map)


#Set items to stay in front
houston_park_cluster_map.keep_in_front(houston_tract_popup_layer,
                                      houston_population_layer,
                                      houston_income_layer,
                                      houston_pop_income_layer,
                                      houston_city_outline_layer,
                                      houston_county_outline_layer
                                      )

#Add layer control
folium.LayerControl(position = 'topright',
                    collapsed = False,
                    autoZIndex = True).add_to(houston_park_cluster_map)

houston_park_cluster_map

In [ ]:
houston_park_cluster_map.save('Houston_Park_Cluster.html')

In [ ]:
#Create map centered on Houston
houston_gym_cluster_map = folium.Map(location=[29.85027,-95.37473],
                   tiles=None, height=700, width=1100, zoom_start=10)

#Clean Tract Outline Layer
houston_tract_popup_layer=folium.GeoJson(houston_gym_park_counts_merge_gjson, 
       name = 'Simple Tract Outlines',
       style_function = lambda feature: {
           'color': 'black',
           'weight': .35, 
           'lineOpacity': 1,
           'fillColor': 'lightgrey',
           'fillOpacity': .8}, 
       show = True,
       overlay = False)
folium.TileLayer(tiles='https://server.arcgisonline.com/ArcGIS/rest/services/Canvas/World_Light_Gray_Base/MapServer/tile/{z}/{y}/{x}', 
                 attr='Tiles &copy; Esri &mdash; Esri, DeLorme, NAVTEQ', 
                 name='Tract Base').add_to(houston_tract_popup_layer)
houston_gym_cluster1 = plugins.MarkerCluster(name='Gym Clusters',
                                            options={'showCoverageOnHover': True,
                                                     'zoomToBoundsOnClick': True,
                                                     'disableClusteringAtZoom': 15,
                                                     'removeOutsideVisibleBounds': True,
                                                     'spiderfyOnMaxZoom': False},
                                            show=False)
for lat, lng, label in zip(houston_gym_filtered.lat, houston_gym_filtered.lng, houston_gym_filtered.name):
    folium.Marker(
        location=[lat, lng],
        icon=folium.Icon(color='white', icon_color='red', icon='heartbeat', prefix='fa'),
        popup=label,
    ).add_to(houston_gym_cluster1)
houston_gym_cluster1.add_to(houston_tract_popup_layer)
houston_tract_popup_layer.add_to(houston_gym_cluster_map)

#Tract Income & Population Density Combined
houston_pop_income_layer=folium.GeoJson(houston_gym_park_counts_merge_gjson, 
       name='Per Capita Income & Population Density', 
       style_function = lambda feature: {
           'color': 'black',
           'weight': 0, 
           'fillColor': houstontractpopulationcolors(feature['properties']['Pop Density per sq km']), 
           'fillOpacity': .5,
           'highlight': True}, 
       overlay = False,
       show = True
        )
folium.GeoJson(houston_gym_park_counts_merge_gjson, 
       style_function = lambda feature: {
           'color': 'black',
           'weight': .35, 
           'fillColor': houstontractincomecolors(feature['properties']['Tract Per Capita Income - 12 Months']), 
           'fillOpacity': .5,
           'highlight': True}, 
       overlay = False
        ).add_to(houston_pop_income_layer)
folium.TileLayer(tiles='https://server.arcgisonline.com/ArcGIS/rest/services/Canvas/World_Light_Gray_Base/MapServer/tile/{z}/{y}/{x}', 
                 attr='Tiles &copy; Esri &mdash; Esri, DeLorme, NAVTEQ', 
                 name='Tract Base').add_to(houston_pop_income_layer)
houston_gym_cluster2 = plugins.MarkerCluster(name='Gym Clusters',
                                            options={'showCoverageOnHover': True,
                                                     'zoomToBoundsOnClick': True,
                                                     'disableClusteringAtZoom': 15,
                                                     'removeOutsideVisibleBounds': True,
                                                     'spiderfyOnMaxZoom': False},
                                            show=False)
for lat, lng, label in zip(houston_gym_filtered.lat, houston_gym_filtered.lng, houston_gym_filtered.name):
    folium.Marker(
        location=[lat, lng],
        icon=folium.Icon(color='white', icon_color='red', icon='heartbeat', prefix='fa'),
        popup=label,
    ).add_to(houston_gym_cluster2)
houston_gym_cluster2.add_to(houston_pop_income_layer)
houston_pop_income_layer.add_to(houston_gym_cluster_map)


#Tract Population Density Color Scale Layer
houston_population_layer=folium.GeoJson(houston_gym_park_counts_merge_gjson, 
       name='Population Density', 
       style_function = lambda feature: {
           'color': 'black',
           'weight': .35, 
           'fillColor': houstontractpopulationcolors(feature['properties']['Pop Density per sq km']), 
           'fillOpacity': .5,
           'highlight': True}, 
       overlay = False
        )
folium.TileLayer(tiles='https://server.arcgisonline.com/ArcGIS/rest/services/Canvas/World_Light_Gray_Base/MapServer/tile/{z}/{y}/{x}', 
                 attr='Tiles &copy; Esri &mdash; Esri, DeLorme, NAVTEQ', 
                 name='Tract Base').add_to(houston_population_layer)
houston_gym_cluster3 = plugins.MarkerCluster(name='Gym Clusters',
                                            options={'showCoverageOnHover': True,
                                                     'zoomToBoundsOnClick': True,
                                                     'disableClusteringAtZoom': 15,
                                                     'removeOutsideVisibleBounds': True,
                                                     'spiderfyOnMaxZoom': False},
                                            show=False)
for lat, lng, label in zip(houston_gym_filtered.lat, houston_gym_filtered.lng, houston_gym_filtered.name):
    folium.Marker(
        location=[lat, lng],
        icon=folium.Icon(color='white', icon_color='red', icon='heartbeat', prefix='fa'),
        popup=label,
    ).add_to(houston_gym_cluster3)
houston_gym_cluster3.add_to(houston_population_layer)
houston_population_layer.add_to(houston_gym_cluster_map)


#Tract Income Color Scale Layer
houston_income_layer=folium.GeoJson(houston_gym_park_counts_merge_gjson, 
       name='Per Capita Income (2019)', 
       style_function = lambda feature: {
           'color': 'black',
           'weight': .35, 
           'fillColor': houstontractincomecolors(feature['properties']['Tract Per Capita Income - 12 Months']), 
           'fillOpacity': .5,
           'highlight': True}, 
       overlay = False
        )
folium.TileLayer(tiles='https://server.arcgisonline.com/ArcGIS/rest/services/Canvas/World_Light_Gray_Base/MapServer/tile/{z}/{y}/{x}', 
                 attr='Tiles &copy; Esri &mdash; Esri, DeLorme, NAVTEQ', 
                 name='Tract Base').add_to(houston_income_layer)
houston_gym_cluster4 = plugins.MarkerCluster(name='Gym Clusters',
                                            options={'showCoverageOnHover': True,
                                                     'zoomToBoundsOnClick': True,
                                                     'disableClusteringAtZoom': 15,
                                                     'removeOutsideVisibleBounds': True,
                                                     'spiderfyOnMaxZoom': False},
                                            show=False)
for lat, lng, label in zip(houston_gym_filtered.lat, houston_gym_filtered.lng, houston_gym_filtered.name):
    folium.Marker(
        location=[lat, lng],
        icon=folium.Icon(color='white', icon_color='red', icon='heartbeat', prefix='fa'),
        popup=label,
    ).add_to(houston_gym_cluster4)
houston_gym_cluster4.add_to(houston_income_layer)
houston_income_layer.add_to(houston_gym_cluster_map)


#County Outline Layer
houston_county_outline_layer=folium.GeoJson(houston_join_county_4326_gjson, 
       name='Surrounding Counties (Purple Outlines)', 
       style_function = lambda feature: {
           'color': 'purple',
           'weight': 2, 
           'fillOpacity': 0,
           'interactive': False
       },
       show=False
       ).add_to(houston_gym_cluster_map)

#City Outline Layer
houston_city_outline_layer=folium.GeoJson(houston_join_place_4326_gjson, 
       name='Houston City Limits (Blue Outline)', 
       style_function = lambda feature: {
           'color': 'blue',
           'weight': 2, 
           'fillOpacity': 0,
           'interactive': False
       },
       show=False
       ).add_to(houston_gym_cluster_map)


#Add Color Scales
houstontractincomecolors.add_to(houston_gym_cluster_map)
houstontractpopulationcolors.add_to(houston_gym_cluster_map)


#Set items to stay in front
houston_gym_cluster_map.keep_in_front(houston_tract_popup_layer,
                                     houston_population_layer,
                                     houston_income_layer,
                                     houston_pop_income_layer,
                                     houston_city_outline_layer,
                                     houston_county_outline_layer
                                     )

#Add layer control
folium.LayerControl(position = 'topright',
                    collapsed = False,
                    autoZIndex = True).add_to(houston_gym_cluster_map)

houston_gym_cluster_map

In [ ]:
houston_gym_cluster_map.save('Houston_Gym_Cluster.html')

##### FourSquare Added in

In [ ]:
#Create map centered on Houston
houston_FS_map = folium.Map(location=[29.85027,-95.37473],
                   tiles=None, height=700, width=1100, zoom_start=10)

#Simple Tract Outline
houston_tract_popup_layer=folium.GeoJson(houston_join_tract_4326_gjson, 
       name='Houston Tract Layer', 
       style_function = lambda feature: {
           'color': 'black',
           'weight': .35, 
           'fillColor': 'white',
           'fillOpacity': 0}, 
       show = True,
       control = False).add_to(houston_FS_map)

#County Outline Layer
houston_county_outline_layer=folium.GeoJson(houston_join_county_4326_gjson, 
       name='Surrounding Counties (Purple Outlines)', 
       style_function = lambda feature: {
           'color': 'purple',
           'weight': 2, 
           'fillOpacity': 0,
           'interactive': False
       },
       show=False
       ).add_to(houston_FS_map)

#City Outline Layer
houston_city_outline_layer=folium.GeoJson(houston_join_place_4326_gjson, 
       name='Houston City Limits (Blue Outline)', 
       style_function = lambda feature: {
           'color': 'blue',
           'weight': 2, 
           'fillOpacity': 0,
           'interactive': False
       },
       show=True
       ).add_to(houston_FS_map)


#Add Foursquare gym search areas
houston_search_radius = folium.FeatureGroup(name='Search Radius', overlay=False, control=True, show=True)
for lat, lng, rad in zip(FS_houston['Tract Center Lat'], FS_houston['Tract Center Long'], FS_houston['Radius']):
    folium.Circle(
        [lat, lng],
        radius=rad,
        color='red',
        stroke=0.5,
        opacity=0.25,
        fill = True,
        fill_color='red',
        fill_opacity=0.5
    ).add_to(houston_search_radius)

folium.TileLayer(tiles='https://server.arcgisonline.com/ArcGIS/rest/services/Canvas/World_Light_Gray_Base/MapServer/tile/{z}/{y}/{x}', 
                 attr='Tiles &copy; Esri &mdash; Esri, DeLorme, NAVTEQ', 
                 name='Tract Base').add_to(houston_search_radius)

houston_search_radius.add_to(houston_FS_map)


#Set items to stay in front
houston_FS_map.keep_in_front(houston_search_radius, houston_tract_popup_layer, houston_county_outline_layer, houston_city_outline_layer)

#Add layer control
folium.LayerControl(position = 'topright',
                    collapsed = False,
                    autoZIndex = True).add_to(houston_FS_map)

houston_FS_map

In [ ]:
houston_FS_map.save('Houston_Search_Radius.html')

##### FourSquare Added in

In [ ]:
#Not created yet

---
## Maps: Set Up

##### *Dallas*
---

In [ ]:
#Look at Quantile values of Income to find a good range for color scale
dallas_min_income, dallas_max_income = dallas_join_tract_4326['Tract Per Capita Income - 12 Months'].quantile([0,.95]).apply(lambda x: round(x, 2))
dallas_mean_income = round(dallas_join_tract_4326['Tract Per Capita Income - 12 Months'].mean(),2)
print(f"Min: {dallas_min_income}", f"Max: {dallas_max_income}", f"Mean: {dallas_mean_income}", sep="\n\n")

In [ ]:
#Set Income Scale
dallastractincomecolors = linear.Greens_05.scale(0,
                                                   100000)
dallastractincomecolors.caption = 'Tract Per Capita Income - Past 12 Months'

In [ ]:
#Look at Quantile values of Population Density to find a good range for color scale
dallas_min_popdense, dallas_max_popdense = dallas_join_tract_4326['Pop Density per sq km'].quantile([0.01,.95]).apply(lambda x: round(x, 2))
dallas_mean_popdense = round(dallas_join_tract_4326['Pop Density per sq km'].mean(),2)
print(f"Min: {dallas_min_popdense}", f"Max: {dallas_max_popdense}", f"Mean: {dallas_mean_popdense}", sep="\n\n")

In [ ]:
#Set Population Density Scale
dallastractpopulationcolors = linear.Reds_05.scale(0,
                                                   5000)
dallastractpopulationcolors.caption = 'Tract Population Density (/sq km)'

In [ ]:
dallas_min_popdense, dallas_max_popdense = dallas_join_tract_4326['Pop Density per sq km'].quantile([0.01,.92]).apply(lambda x: round(x, 2))
dallas_mean_popdense = round(dallas_join_tract_4326['Pop Density per sq km'].mean(),2)
print(f"Min: {dallas_min_popdense}", f"Max: {dallas_max_popdense}", f"Mean: {dallas_mean_popdense}", sep="\n\n")

---
## Maps: Generation 

##### *Dallas*
---

In [ ]:
print(dallas_place['Place Center Lat'], dallas_place['Place Center Long'])

In [ ]:
#Create map centered on Dallas
final_dallas_map = folium.Map(location=[32.8033334, -96.8665128],
                   tiles=None, height=700, width=1000, zoom_start=10)

#Clean Popup Layer
dallas_tract_popup_layer=folium.GeoJson(dallas_gym_park_counts_merge_gjson, 
       name = 'Simple Tract Outlines',
       style_function = lambda feature: {
           'color': 'black',
           'weight': .35, 
           'lineOpacity': 1,
           'fillColor': 'lightgrey',
           'fillOpacity': .8}, 
       highlight_function=lambda x: {
            'color': 'black',
            'weight': 2,
            'lineOpacity': .8,
            'fillColor': 'grey',
            'fillOpacity': .5},
       tooltip = folium.GeoJsonTooltip(fields=('Tract Area Name', 
                                               'Tract Per Capita Income - 12 Months',
                                               'Pop Density per sq km',
                                               'Number of Parks',
                                               'Number of Gyms',
                                               'Tract Percentage White',
                                               'Tract Percentage Black or African American',
                                               'Tract Percentage Asian',
                                               'Tract Percentage American Indian and Alaskan Native',
                                               'Tract Percentage Other Race'),
                                       aliases=('Census Tract (#, County Name, State):',
                                                'Tract Income (2019 Inflation Adjusted USD):',
                                                'Tract Population Density (/sq km):',
                                                'Number of Parks:',
                                                'Number of Gyms:',
                                                'Population % White:',
                                                'Population % Black/African American:',
                                                'Population % Asian:',
                                                'Population % American Indian',
                                                'Population % Other'),
                                       sticky=True,
                                       localize=True),
       show = True,
       overlay = False,
       zoom_on_click = True)
folium.TileLayer(tiles='https://server.arcgisonline.com/ArcGIS/rest/services/Canvas/World_Light_Gray_Base/MapServer/tile/{z}/{y}/{x}', 
                 attr='Tiles &copy; Esri &mdash; Esri, DeLorme, NAVTEQ', 
                 name='Tract Base').add_to(dallas_tract_popup_layer)
dallas_tract_popup_layer.add_to(final_dallas_map)


#Tract Income & Population Density Combined
dallas_pop_income_layer=folium.GeoJson(dallas_gym_park_counts_merge_gjson, 
       name='Per Capita Income & Population Density', 
       style_function = lambda feature: {
           'color': 'black',
           'weight': 0, 
           'fillColor': dallastractpopulationcolors(feature['properties']['Pop Density per sq km']), 
           'fillOpacity': .5,
           'highlight': True}, 
       overlay = False,
       show = True
        )
folium.GeoJson(dallas_gym_park_counts_merge_gjson, 
       style_function = lambda feature: {
           'color': 'black',
           'weight': 0, 
           'fillColor': dallastractincomecolors(feature['properties']['Tract Per Capita Income - 12 Months']), 
           'fillOpacity': .5,
           'highlight': True}, 
       overlay = False
        ).add_to(dallas_pop_income_layer)
folium.TileLayer(tiles='https://server.arcgisonline.com/ArcGIS/rest/services/Canvas/World_Light_Gray_Base/MapServer/tile/{z}/{y}/{x}', 
                 attr='Tiles &copy; Esri &mdash; Esri, DeLorme, NAVTEQ', 
                 name='Tract Base').add_to(dallas_pop_income_layer)
dallas_tract_popup_layer1=folium.GeoJson(dallas_gym_park_counts_merge_gjson, 
       style_function = lambda feature: {
           'color': 'black',
           'weight': .35, 
           'fillColor': 'white',
           'fillOpacity': 0}, 
       highlight_function=lambda x: {
            'color': 'black',
            'weight': 2,
            'lineOpacity': .8,
            'fillColor': 'grey',
            'fillOpacity': .25},
       tooltip = folium.GeoJsonTooltip(fields=('Tract Area Name', 
                                               'Tract Per Capita Income - 12 Months',
                                               'Pop Density per sq km',
                                               'Number of Parks',
                                               'Number of Gyms',
                                               'Tract Percentage White',
                                               'Tract Percentage Black or African American',
                                               'Tract Percentage Asian',
                                               'Tract Percentage American Indian and Alaskan Native',
                                               'Tract Percentage Other Race'),
                                       aliases=('Census Tract (#, County Name, State):',
                                                'Tract Income (2019 Inflation Adjusted USD):',
                                                'Tract Population Density (/sq km):',
                                                'Number of Parks:',
                                                'Number of Gyms:',
                                                'Population % White:',
                                                'Population % Black/African American:',
                                                'Population % Asian:',
                                                'Population % American Indian',
                                                'Population % Other'),
                                       sticky=True,
                                       localize=True),
       show = True,
       overlay = False,
       zoom_on_click = True).add_to(dallas_pop_income_layer)
dallas_pop_income_layer.add_to(final_dallas_map)


#Tract Population Density Color Scale Layer
dallas_population_layer=folium.GeoJson(dallas_gym_park_counts_merge_gjson, 
       name='Population Density', 
       style_function = lambda feature: {
           'color': 'black',
           'weight': 0, 
           'fillColor': dallastractpopulationcolors(feature['properties']['Pop Density per sq km']), 
           'fillOpacity': .5,
           'highlight': True}, 
       overlay = False
        )
folium.TileLayer(tiles='https://server.arcgisonline.com/ArcGIS/rest/services/Canvas/World_Light_Gray_Base/MapServer/tile/{z}/{y}/{x}', 
                 attr='Tiles &copy; Esri &mdash; Esri, DeLorme, NAVTEQ', 
                 name='Tract Base').add_to(dallas_population_layer)
dallas_tract_popup_layer2=folium.GeoJson(dallas_gym_park_counts_merge_gjson, 
       style_function = lambda feature: {
           'color': 'black',
           'weight': .35, 
           'fillColor': 'white',
           'fillOpacity': 0}, 
       highlight_function=lambda x: {
            'color': 'black',
            'weight': 2,
            'lineOpacity': .8,
            'fillColor': 'grey',
            'fillOpacity': .25},
       tooltip = folium.GeoJsonTooltip(fields=('Tract Area Name', 
                                               'Tract Per Capita Income - 12 Months',
                                               'Pop Density per sq km',
                                               'Number of Parks',
                                               'Number of Gyms',
                                               'Tract Percentage White',
                                               'Tract Percentage Black or African American',
                                               'Tract Percentage Asian',
                                               'Tract Percentage American Indian and Alaskan Native',
                                               'Tract Percentage Other Race'),
                                       aliases=('Census Tract (#, County Name, State):',
                                                'Tract Income (2019 Inflation Adjusted USD):',
                                                'Tract Population Density (/sq km):',
                                                'Number of Parks:',
                                                'Number of Gyms:',
                                                'Population % White:',
                                                'Population % Black/African American:',
                                                'Population % Asian:',
                                                'Population % American Indian',
                                                'Population % Other'),
                                       sticky=True,
                                       localize=True),
       show = True,
       overlay = False,
       zoom_on_click = True).add_to(dallas_population_layer)
dallas_population_layer.add_to(final_dallas_map)


#Tract Income Color Scale Layer
dallas_income_layer=folium.GeoJson(dallas_gym_park_counts_merge_gjson, 
       name='Per Capita Income (2019)', 
       style_function = lambda feature: {
           'color': 'black',
           'weight': 0, 
           'fillColor': dallastractincomecolors(feature['properties']['Tract Per Capita Income - 12 Months']), 
           'fillOpacity': .5,
           'highlight': True}, 
       overlay = False
        )
folium.TileLayer(tiles='https://server.arcgisonline.com/ArcGIS/rest/services/Canvas/World_Light_Gray_Base/MapServer/tile/{z}/{y}/{x}', 
                 attr='Tiles &copy; Esri &mdash; Esri, DeLorme, NAVTEQ', 
                 name='Tract Base').add_to(dallas_income_layer)
dallas_tract_popup_layer3=folium.GeoJson(dallas_gym_park_counts_merge_gjson,  
       style_function = lambda feature: {
           'color': 'black',
           'weight': .35, 
           'fillColor': 'white',
           'fillOpacity': 0}, 
       highlight_function=lambda x: {
            'color': 'black',
            'weight': 2,
            'lineOpacity': .8,
            'fillColor': 'grey',
            'fillOpacity': .25},
       tooltip = folium.GeoJsonTooltip(fields=('Tract Area Name', 
                                               'Tract Per Capita Income - 12 Months',
                                               'Pop Density per sq km',
                                               'Number of Parks',
                                               'Number of Gyms',
                                               'Tract Percentage White',
                                               'Tract Percentage Black or African American',
                                               'Tract Percentage Asian',
                                               'Tract Percentage American Indian and Alaskan Native',
                                               'Tract Percentage Other Race'),
                                       aliases=('Census Tract (#, County Name, State):',
                                                'Tract Income (2019 Inflation Adjusted USD):',
                                                'Tract Population Density (/sq km):',
                                                'Number of Parks:',
                                                'Number of Gyms:',
                                                'Population % White:',
                                                'Population % Black/African American:',
                                                'Population % Asian:',
                                                'Population % American Indian',
                                                'Population % Other'),
                                       sticky=True,
                                       localize=True),
       show = True,
       overlay = False,
       zoom_on_click = True).add_to(dallas_income_layer)
dallas_income_layer.add_to(final_dallas_map)


#County Outline Layer
dallas_county_outline_layer=folium.GeoJson(dallas_join_county_4326_gjson, 
       name='Surrounding Counties (Purple Outlines)', 
       style_function = lambda feature: {
           'color': 'purple',
           'weight': 2, 
           'fillOpacity': 0,
           'interactive': False
       },
       show=False
       ).add_to(final_dallas_map)

#City Outline Layer
dallas_city_outline_layer=folium.GeoJson(dallas_join_place_4326_gjson, 
       name='Dallas City Limits (Blue Outline)', 
       style_function = lambda feature: {
           'color': 'blue',
           'weight': 2, 
           'fillOpacity': 0,
           'interactive': False
       },
       show=False
       ).add_to(final_dallas_map)

#Add Foursquare gym results as markers
dallas_gym_markers = folium.FeatureGroup(name='Gym Locations (Yellow Dots)', overlay=True, control=True, show=False)
for lat, lng, label in zip(dallas_gym_filtered.lat, dallas_gym_filtered.lng, dallas_gym_filtered.name):
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        color='gray',
        weight=1,
        popup=label,
        fill = True,
        fill_color='yellow',
        fill_opacity=0.8
    ).add_to(dallas_gym_markers)
dallas_gym_markers.add_to(final_dallas_map)

#Add Foursquare gym results as markers
dallas_park_markers = folium.FeatureGroup(name='Park Locations (Blue Dots)', overlay=True, control=True, show=False)
for lat, lng, label in zip(dallas_park_filtered.lat, dallas_park_filtered.lng, dallas_park_filtered.name):
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        color='gray',
        weight=1,
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.8,
    ).add_to(dallas_park_markers)
dallas_park_markers.add_to(final_dallas_map)


#Add Color Scales
dallastractincomecolors.add_to(final_dallas_map)
dallastractpopulationcolors.add_to(final_dallas_map)


#Set order of layers
final_dallas_map.keep_in_front(dallas_tract_popup_layer,
                               dallas_population_layer,
                               dallas_income_layer,
                               dallas_pop_income_layer,
                               dallas_city_outline_layer,
                               dallas_county_outline_layer,
                               dallas_gym_markers,
                               dallas_park_markers
                               )

#Add layer control
folium.LayerControl(position = 'topleft',
                    collapsed = False,
                    autoZIndex = True).add_to(final_dallas_map)

final_dallas_map

In [ ]:
final_dallas_map.save('Dallas_Map.html')

In [ ]:
#Create map centered on Dallas
dallas_park_cluster_map = folium.Map(location=[32.8033334, -96.8665128],
                   tiles=None, height=700, width=1000, zoom_start=10)

#Clean Tract Outline Layer
dallas_tract_popup_layer=folium.GeoJson(dallas_gym_park_counts_merge_gjson, 
       name = 'Simple Tract Outlines',
       style_function = lambda feature: {
           'color': 'black',
           'weight': .35, 
           'lineOpacity': 1,
           'fillColor': 'lightgrey',
           'fillOpacity': .8}, 
       show = True,
       overlay = False)
folium.TileLayer(tiles='https://server.arcgisonline.com/ArcGIS/rest/services/Canvas/World_Light_Gray_Base/MapServer/tile/{z}/{y}/{x}', 
                 attr='Tiles &copy; Esri &mdash; Esri, DeLorme, NAVTEQ', 
                 name='Tract Base').add_to(dallas_tract_popup_layer)
dallas_park_cluster1 = plugins.MarkerCluster(name='Park Clusters',
                                            options={'showCoverageOnHover': True,
                                                     'zoomToBoundsOnClick': True,
                                                     'disableClusteringAtZoom': 15,
                                                     'removeOutsideVisibleBounds': True,
                                                     'spiderfyOnMaxZoom': False},
                                            show=False)
for lat, lng, label in zip(dallas_park_filtered.lat, dallas_park_filtered.lng, dallas_park_filtered.name):
    folium.Marker(
        location=[lat, lng],
        icon=folium.Icon(color='green', icon_color='white', icon='tree', prefix='fa'),
        popup=label,
    ).add_to(dallas_park_cluster1)
dallas_park_cluster1.add_to(dallas_tract_popup_layer)
dallas_tract_popup_layer.add_to(dallas_park_cluster_map)

#Tract Income & Population Density Combined
dallas_pop_income_layer=folium.GeoJson(dallas_gym_park_counts_merge_gjson, 
       name='Per Capita Income & Population Density', 
       style_function = lambda feature: {
           'color': 'black',
           'weight': 0, 
           'fillColor': dallastractpopulationcolors(feature['properties']['Pop Density per sq km']), 
           'fillOpacity': .5,
           'highlight': True}, 
       overlay = False,
       show = True
        )
folium.GeoJson(dallas_gym_park_counts_merge_gjson, 
       style_function = lambda feature: {
           'color': 'black',
           'weight': .35, 
           'fillColor': dallastractincomecolors(feature['properties']['Tract Per Capita Income - 12 Months']), 
           'fillOpacity': .5,
           'highlight': True}, 
       overlay = False
        ).add_to(dallas_pop_income_layer)
folium.TileLayer(tiles='https://server.arcgisonline.com/ArcGIS/rest/services/Canvas/World_Light_Gray_Base/MapServer/tile/{z}/{y}/{x}', 
                 attr='Tiles &copy; Esri &mdash; Esri, DeLorme, NAVTEQ', 
                 name='Tract Base').add_to(dallas_pop_income_layer)
dallas_park_cluster2 = plugins.MarkerCluster(name='Park Clusters',
                                            options={'showCoverageOnHover': True,
                                                     'zoomToBoundsOnClick': True,
                                                     'disableClusteringAtZoom': 15,
                                                     'removeOutsideVisibleBounds': True,
                                                     'spiderfyOnMaxZoom': False},
                                            show=False)
for lat, lng, label in zip(dallas_park_filtered.lat, dallas_park_filtered.lng, dallas_park_filtered.name):
    folium.Marker(
        location=[lat, lng],
        icon=folium.Icon(color='green', icon_color='white', icon='tree', prefix='fa'),
        popup=label,
    ).add_to(dallas_park_cluster2)
dallas_park_cluster2.add_to(dallas_pop_income_layer)
dallas_pop_income_layer.add_to(dallas_park_cluster_map)


#Tract Population Density Color Scale Layer
dallas_population_layer=folium.GeoJson(dallas_gym_park_counts_merge_gjson, 
       name='Population Density', 
       style_function = lambda feature: {
           'color': 'black',
           'weight': .35, 
           'fillColor': dallastractpopulationcolors(feature['properties']['Pop Density per sq km']), 
           'fillOpacity': .5,
           'highlight': True}, 
       overlay = False
        )
folium.TileLayer(tiles='https://server.arcgisonline.com/ArcGIS/rest/services/Canvas/World_Light_Gray_Base/MapServer/tile/{z}/{y}/{x}', 
                 attr='Tiles &copy; Esri &mdash; Esri, DeLorme, NAVTEQ', 
                 name='Tract Base').add_to(dallas_population_layer)
dallas_park_cluster3 = plugins.MarkerCluster(name='Park Clusters',
                                            options={'showCoverageOnHover': True,
                                                     'zoomToBoundsOnClick': True,
                                                     'disableClusteringAtZoom': 15,
                                                     'removeOutsideVisibleBounds': True,
                                                     'spiderfyOnMaxZoom': False},
                                            show=False)
for lat, lng, label in zip(dallas_park_filtered.lat, dallas_park_filtered.lng, dallas_park_filtered.name):
    folium.Marker(
        location=[lat, lng],
        icon=folium.Icon(color='green', icon_color='white', icon='tree', prefix='fa'),
        popup=label,
    ).add_to(dallas_park_cluster3)
dallas_park_cluster3.add_to(dallas_population_layer)
dallas_population_layer.add_to(dallas_park_cluster_map)


#Tract Income Color Scale Layer
dallas_income_layer=folium.GeoJson(dallas_gym_park_counts_merge_gjson, 
       name='Per Capita Income (2019)', 
       style_function = lambda feature: {
           'color': 'black',
           'weight': .35, 
           'fillColor': dallastractincomecolors(feature['properties']['Tract Per Capita Income - 12 Months']), 
           'fillOpacity': .5,
           'highlight': True}, 
       overlay = False
        )
folium.TileLayer(tiles='https://server.arcgisonline.com/ArcGIS/rest/services/Canvas/World_Light_Gray_Base/MapServer/tile/{z}/{y}/{x}', 
                 attr='Tiles &copy; Esri &mdash; Esri, DeLorme, NAVTEQ', 
                 name='Tract Base').add_to(dallas_income_layer)
dallas_park_cluster4 = plugins.MarkerCluster(name='Park Clusters',
                                            options={'showCoverageOnHover': True,
                                                     'zoomToBoundsOnClick': True,
                                                     'disableClusteringAtZoom': 15,
                                                     'removeOutsideVisibleBounds': True,
                                                     'spiderfyOnMaxZoom': False},
                                            show=False)
for lat, lng, label in zip(dallas_park_filtered.lat, dallas_park_filtered.lng, dallas_park_filtered.name):
    folium.Marker(
        location=[lat, lng],
        icon=folium.Icon(color='green', icon_color='white', icon='tree', prefix='fa'),
        popup=label,
    ).add_to(dallas_park_cluster4)
dallas_park_cluster4.add_to(dallas_income_layer)
dallas_income_layer.add_to(dallas_park_cluster_map)


#County Outline Layer
dallas_county_outline_layer=folium.GeoJson(dallas_join_county_4326_gjson, 
       name='Surrounding Counties (Purple Outlines)', 
       style_function = lambda feature: {
           'color': 'purple',
           'weight': 2, 
           'fillOpacity': 0,
           'interactive': False
       },
       show=False
       ).add_to(dallas_park_cluster_map)

#City Outline Layer
dallas_city_outline_layer=folium.GeoJson(dallas_join_place_4326_gjson, 
       name='Dallas City Limits (Blue Outline)', 
       style_function = lambda feature: {
           'color': 'blue',
           'weight': 2, 
           'fillOpacity': 0,
           'interactive': False
       },
       show=False
       ).add_to(dallas_park_cluster_map)


#Add Color Scales
dallastractincomecolors.add_to(dallas_park_cluster_map)
dallastractpopulationcolors.add_to(dallas_park_cluster_map)


#Set items to stay in front
dallas_park_cluster_map.keep_in_front(dallas_tract_popup_layer,
                                      dallas_population_layer,
                                      dallas_income_layer,
                                      dallas_pop_income_layer,
                                      dallas_city_outline_layer,
                                      dallas_county_outline_layer
                                      )

#Add layer control
folium.LayerControl(position = 'topleft',
                    collapsed = False).add_to(dallas_park_cluster_map)

dallas_park_cluster_map

In [ ]:
dallas_park_cluster_map.save('Dallas_Park_Cluster.html')

In [ ]:
#Create map centered on Dallas
dallas_gym_cluster_map = folium.Map(location=[32.8033334, -96.8665128],
                   tiles=None, height=700, width=1000, zoom_start=10)

#Clean Tract Outline Layer
dallas_tract_popup_layer=folium.GeoJson(dallas_gym_park_counts_merge_gjson, 
       name = 'Simple Tract Outlines',
       style_function = lambda feature: {
           'color': 'black',
           'weight': .35, 
           'lineOpacity': 1,
           'fillColor': 'lightgrey',
           'fillOpacity': .8}, 
       show = True,
       overlay = False)
folium.TileLayer(tiles='https://server.arcgisonline.com/ArcGIS/rest/services/Canvas/World_Light_Gray_Base/MapServer/tile/{z}/{y}/{x}', 
                 attr='Tiles &copy; Esri &mdash; Esri, DeLorme, NAVTEQ', 
                 name='Tract Base').add_to(dallas_tract_popup_layer)
dallas_gym_cluster1 = plugins.MarkerCluster(name='Gym Clusters',
                                            options={'showCoverageOnHover': True,
                                                     'zoomToBoundsOnClick': True,
                                                     'disableClusteringAtZoom': 15,
                                                     'removeOutsideVisibleBounds': True,
                                                     'spiderfyOnMaxZoom': False},
                                            show=False)
for lat, lng, label in zip(dallas_gym_filtered.lat, dallas_gym_filtered.lng, dallas_gym_filtered.name):
    folium.Marker(
        location=[lat, lng],
        icon=folium.Icon(color='white', icon_color='red', icon='heartbeat', prefix='fa'),
        popup=label,
    ).add_to(dallas_gym_cluster1)
dallas_gym_cluster1.add_to(dallas_tract_popup_layer)
dallas_tract_popup_layer.add_to(dallas_gym_cluster_map)

#Tract Income & Population Density Combined
dallas_pop_income_layer=folium.GeoJson(dallas_gym_park_counts_merge_gjson, 
       name='Per Capita Income & Population Density', 
       style_function = lambda feature: {
           'color': 'black',
           'weight': 0, 
           'fillColor': dallastractpopulationcolors(feature['properties']['Pop Density per sq km']), 
           'fillOpacity': .5,
           'highlight': True}, 
       overlay = False,
       show = True
        )
folium.GeoJson(dallas_gym_park_counts_merge_gjson, 
       style_function = lambda feature: {
           'color': 'black',
           'weight': .35, 
           'fillColor': dallastractincomecolors(feature['properties']['Tract Per Capita Income - 12 Months']), 
           'fillOpacity': .5,
           'highlight': True}, 
       overlay = False
        ).add_to(dallas_pop_income_layer)
folium.TileLayer(tiles='https://server.arcgisonline.com/ArcGIS/rest/services/Canvas/World_Light_Gray_Base/MapServer/tile/{z}/{y}/{x}', 
                 attr='Tiles &copy; Esri &mdash; Esri, DeLorme, NAVTEQ', 
                 name='Tract Base').add_to(dallas_pop_income_layer)
dallas_gym_cluster2 = plugins.MarkerCluster(name='Gym Clusters',
                                            options={'showCoverageOnHover': True,
                                                     'zoomToBoundsOnClick': True,
                                                     'disableClusteringAtZoom': 15,
                                                     'removeOutsideVisibleBounds': True,
                                                     'spiderfyOnMaxZoom': False},
                                            show=False)
for lat, lng, label in zip(dallas_gym_filtered.lat, dallas_gym_filtered.lng, dallas_gym_filtered.name):
    folium.Marker(
        location=[lat, lng],
        icon=folium.Icon(color='white', icon_color='red', icon='heartbeat', prefix='fa'),
        popup=label,
    ).add_to(dallas_gym_cluster2)
dallas_gym_cluster2.add_to(dallas_pop_income_layer)
dallas_pop_income_layer.add_to(dallas_gym_cluster_map)


#Tract Population Density Color Scale Layer
dallas_population_layer=folium.GeoJson(dallas_gym_park_counts_merge_gjson, 
       name='Population Density', 
       style_function = lambda feature: {
           'color': 'black',
           'weight': .35, 
           'fillColor': dallastractpopulationcolors(feature['properties']['Pop Density per sq km']), 
           'fillOpacity': .5,
           'highlight': True}, 
       overlay = False
        )
folium.TileLayer(tiles='https://server.arcgisonline.com/ArcGIS/rest/services/Canvas/World_Light_Gray_Base/MapServer/tile/{z}/{y}/{x}', 
                 attr='Tiles &copy; Esri &mdash; Esri, DeLorme, NAVTEQ', 
                 name='Tract Base').add_to(dallas_population_layer)
dallas_gym_cluster3 = plugins.MarkerCluster(name='Gym Clusters',
                                            options={'showCoverageOnHover': True,
                                                     'zoomToBoundsOnClick': True,
                                                     'disableClusteringAtZoom': 15,
                                                     'removeOutsideVisibleBounds': True,
                                                     'spiderfyOnMaxZoom': False},
                                            show=False)
for lat, lng, label in zip(dallas_gym_filtered.lat, dallas_gym_filtered.lng, dallas_gym_filtered.name):
    folium.Marker(
        location=[lat, lng],
        icon=folium.Icon(color='white', icon_color='red', icon='heartbeat', prefix='fa'),
        popup=label,
    ).add_to(dallas_gym_cluster3)
dallas_gym_cluster3.add_to(dallas_population_layer)
dallas_population_layer.add_to(dallas_gym_cluster_map)


#Tract Income Color Scale Layer
dallas_income_layer=folium.GeoJson(dallas_gym_park_counts_merge_gjson, 
       name='Per Capita Income (2019)', 
       style_function = lambda feature: {
           'color': 'black',
           'weight': .35, 
           'fillColor': dallastractincomecolors(feature['properties']['Tract Per Capita Income - 12 Months']), 
           'fillOpacity': .5,
           'highlight': True}, 
       overlay = False
        )
folium.TileLayer(tiles='https://server.arcgisonline.com/ArcGIS/rest/services/Canvas/World_Light_Gray_Base/MapServer/tile/{z}/{y}/{x}', 
                 attr='Tiles &copy; Esri &mdash; Esri, DeLorme, NAVTEQ', 
                 name='Tract Base').add_to(dallas_income_layer)
dallas_gym_cluster4 = plugins.MarkerCluster(name='Gym Clusters',
                                            options={'showCoverageOnHover': True,
                                                     'zoomToBoundsOnClick': True,
                                                     'disableClusteringAtZoom': 15,
                                                     'removeOutsideVisibleBounds': True,
                                                     'spiderfyOnMaxZoom': False},
                                            show=False)
for lat, lng, label in zip(dallas_gym_filtered.lat, dallas_gym_filtered.lng, dallas_gym_filtered.name):
    folium.Marker(
        location=[lat, lng],
        icon=folium.Icon(color='white', icon_color='red', icon='heartbeat', prefix='fa'),
        popup=label,
    ).add_to(dallas_gym_cluster4)
dallas_gym_cluster4.add_to(dallas_income_layer)
dallas_income_layer.add_to(dallas_gym_cluster_map)


#County Outline Layer
dallas_county_outline_layer=folium.GeoJson(dallas_join_county_4326_gjson, 
       name='Surrounding Counties (Purple Outlines)', 
       style_function = lambda feature: {
           'color': 'purple',
           'weight': 2, 
           'fillOpacity': 0,
           'interactive': False
       },
       show=False
       ).add_to(dallas_gym_cluster_map)

#City Outline Layer
dallas_city_outline_layer=folium.GeoJson(dallas_join_place_4326_gjson, 
       name='Dallas City Limits (Blue Outline)', 
       style_function = lambda feature: {
           'color': 'blue',
           'weight': 2, 
           'fillOpacity': 0,
           'interactive': False
       },
       show=False
       ).add_to(dallas_gym_cluster_map)


#Add Color Scales
dallastractincomecolors.add_to(dallas_gym_cluster_map)
dallastractpopulationcolors.add_to(dallas_gym_cluster_map)


#Set items to stay in front
dallas_gym_cluster_map.keep_in_front(dallas_tract_popup_layer,
                                     dallas_population_layer,
                                     dallas_income_layer,
                                     dallas_pop_income_layer,
                                     dallas_city_outline_layer,
                                     dallas_county_outline_layer
                                     )

#Add layer control
folium.LayerControl(position = 'topleft',
                    collapsed = False).add_to(dallas_gym_cluster_map)

dallas_gym_cluster_map

In [ ]:
dallas_gym_cluster_map.save('Dallas_Gym_Cluster.html')

##### FourSquare Added in

In [ ]:
#Create map centered on Dallas
dallas_FS_map = folium.Map(location=[32.8033334, -96.8665128],
                   tiles=None, height=700, width=1000, zoom_start=10)
#Simple Tract Outline
dallas_tract_popup_layer=folium.GeoJson(dallas_join_tract_4326_gjson, 
       name='Dallas Tract Layer', 
       style_function = lambda feature: {
           'color': 'black',
           'weight': .35, 
           'fillColor': 'white',
           'fillOpacity': 0}, 
       show = True,
       control = False).add_to(dallas_FS_map)

#County Outline Layer
dallas_county_outline_layer=folium.GeoJson(dallas_join_county_4326_gjson, 
       name='Surrounding Counties (Purple Outlines)', 
       style_function = lambda feature: {
           'color': 'purple',
           'weight': 2, 
           'fillOpacity': 0,
           'interactive': False
       },
       show=False
       ).add_to(dallas_FS_map)

#City Outline Layer
dallas_city_outline_layer=folium.GeoJson(dallas_join_place_4326_gjson, 
       name='Dallas City Limits (Blue Outline)', 
       style_function = lambda feature: {
           'color': 'blue',
           'weight': 2, 
           'fillOpacity': 0,
           'interactive': False
       },
       show=True
       ).add_to(dallas_FS_map)


#Add Foursquare gym search areas
dallas_search_radius = folium.FeatureGroup(name='Search Radius', overlay=False, control=True, show=True)
for lat, lng, rad in zip(FS_dallas['Tract Center Lat'], FS_dallas['Tract Center Long'], FS_dallas['Radius']):
    folium.Circle(
        [lat, lng],
        radius=rad,
        color='red',
        stroke=0.5,
        opacity=0.25,
        fill = True,
        fill_color='red',
        fill_opacity=0.5
    ).add_to(dallas_search_radius)

folium.TileLayer(tiles='https://server.arcgisonline.com/ArcGIS/rest/services/Canvas/World_Light_Gray_Base/MapServer/tile/{z}/{y}/{x}', 
                 attr='Tiles &copy; Esri &mdash; Esri, DeLorme, NAVTEQ', 
                 name='Tract Base').add_to(dallas_search_radius)

dallas_search_radius.add_to(dallas_FS_map)


#Set items to stay in front
dallas_FS_map.keep_in_front(dallas_search_radius, dallas_tract_popup_layer, dallas_county_outline_layer, dallas_city_outline_layer)

#Add layer control
folium.LayerControl(position = 'topleft',
                    collapsed = False).add_to(dallas_FS_map)

dallas_FS_map

In [ ]:
dallas_FS_map.save('Dallas_Search_Radius.html')

In [ ]:
#import matplotlib.pyplot as plt
## Pie chart
#races = ['Population % White',
#         'Population % Black or African American',
#         'Population % Asian',
#         'Population % American Indian and Alaskan Native',
#         'Population % Other Race']
#sizes = [dallas_gym_park_counts_merge_gjson['Tract Percentage White'],
#         dallas_gym_park_counts_merge_gjson['Tract Percentage Black or African American'],
#         dallas_gym_park_counts_merge_gjson['Tract Percentage Asian'],
#         dallas_gym_park_counts_merge_gjson['Tract Percentage American Indian and Alaskan Native'],
#         dallas_gym_park_counts_merge_gjson['Tract Percentage Other Race']]
##colors
#colors = ['#ff9999','#66b3ff','#99ff99','#ffcc99','#ffb3e6']
##explsion
#explode = (0.05,0.05,0.05,0.05,0.05)
 
#plot.pie(sizes, colors = colors, labels=races, autopct='%1.1f%%', startangle=90, pctdistance=0.85, explode = explode)
##draw circle
#centre_circle = plt.Circle((0,0),0.70,fc='white')
#fig = plt.gcf()
#fig.gca().add_artist(centre_circle)
## Equal aspect ratio ensures that pie is drawn as a circle
#ax1.axis('equal')  
#plt.tight_layout()
#plt.show()

In [ ]:
#DALLAS_FS1 = %sql select * from DALLAS_FS1; 
#dallas_fs_test = DALLAS_FS1.DataFrame()
#dallas_fs_test.head()

In [ ]:
#dallas_fs_test['location_lat'] = pd.to_numeric(dallas_fs_test['location_lat'],errors='coerce')
#dallas_fs_test['location_lng'] = pd.to_numeric(dallas_fs_test['location_lng'],errors='coerce')

In [ ]:
#geometry = [Point(xy) for xy in zip(dallas_fs_test.location_lng, dallas_fs_test.location_lat)]
#dallas_fs_test_clean = dallas_fs_test
#dallas_fs_test_clean_gdf = GeoDataFrame(dallas_fs_test_clean, crs="EPSG:4326", geometry=geometry)

#dallas_fs_test_intersect = gpd.sjoin(dallas_fs_test_clean_gdf,dallas_place_4326,how='inner',op='within')

In [ ]:
##gym markers
#dallasgyms = plugins.MarkerCluster().add_to(dallas_fs_map)

## loop through the dataframe and add each data point to the mark cluster
#for lat, lng, name, in zip(dallas_fs_test_intersect.location_lat, dallas_fs_test_intersect.location_lng, dallas_fs_test_intersect.name):
#    folium.Marker(
#        location=[lat, lng],
#        icon=None,
#        popup=name,
#    ).add_to(dallasgyms)